In [1]:
#Running the sanity checked code from 
# /Users/marcellamirabelli/Desktop/rbp_localization_project/Tiling_Version3_V1.ipynb on all the test proteins 

Cell 1: 


Goal of this code block: 
to pull the uniprot IDs from the "df_master.pkl" file and loads sequences. 
ID truth is where we have a list of the uniprot ID and canonical truth label of its localization (but will validatae also later)


Extra IDs: where you can add IDs that dont have a truth label if you want to score them anyway, I have no used this yet
Thresholds: these filters are applied later but this sets them up. For this the disorder threshold is 0.5, where a tile is only kept if it has a mean disorder of >/ 0.5 and a tile is only kept if it has a ProtGPS score of >/ 0.65
 

Then it calls for the metapredict env name to use the metapredict model, and same with prot gps, and makes the bath sizes 8 tiles per batch

This block also lays out the parameters for stage 1 and 2 of tiling

In [2]:
# ===== Cell 1: Config + Run folder =====
from pathlib import Path   # for path manipulations
from datetime import datetime # for timestamped run folder

# ---- Data / IDs ----
DF_MASTER_PATH = "/Users/marcellamirabelli/Desktop/rbp_localization_project/data/df_master.pkl" # master dataframe path
ID_TRUTH = [ # tuples of (uniprotID, localization) for test proteins
     # test protein 
   # --- Chromosome ---
("Q9C0C2", "chromosome"), ("Q8NEY8", "chromosome"), ("P46013", "chromosome"),
("Q96SB4", "chromosome"), ("Q14684", "chromosome"), ("Q5UIP0", "chromosome"),

# --- Nuclear speckle ---
("Q13427", "nuclear_speckle"), ("Q7L4I2", "nuclear_speckle"), ("Q9Y383", "nuclear_speckle"),
("O75400", "nuclear_speckle"), ("O95232", "nuclear_speckle"), ("P18583", "nuclear_speckle"),
("Q01130", "nuclear_speckle"), ("Q02040", "nuclear_speckle"), ("Q13435", "nuclear_speckle"),
("Q14004", "nuclear_speckle"), ("Q14498", "nuclear_speckle"), ("Q16630", "nuclear_speckle"),
("Q6PJT7", "nuclear_speckle"), ("Q7L014", "nuclear_speckle"), ("Q8WUQ7", "nuclear_speckle"),
("Q9NYF8", "nuclear_speckle"), ("Q9UQ35", "nuclear_speckle"), ("Q9Y2W1", "nuclear_speckle"),

# --- Nucleolus ---
("P19338", "nucleolus"), ("Q9BSC4", "nucleolus"), ("P46087", "nucleolus"),
("Q8N8A6", "nucleolus"), ("Q13428", "nucleolus"), ("Q14978", "nucleolus"),
("P06748", "nucleolus"), ("O00567", "nucleolus"), ("O60832", "nucleolus"),
("P22087", "nucleolus"), ("P27695", "nucleolus"), ("Q13206", "nucleolus"),
("Q8TA86", "nucleolus"), ("Q9NR30", "nucleolus"), ("Q9NY12", "nucleolus"),

# --- P-body ---
("Q9UPQ9", "p-body"), ("Q8NDV7", "p-body"), ("P26196", "p-body"),
("Q92900", "p-body"), ("Q14671", "p-body"), ("Q96EP5", "p-body"),

# --- Stress granule ---
("O00571", "stress_granule"), ("Q71RC2", "stress_granule"), ("Q9BYJ9", "stress_granule"),
("Q9Y5A9", "stress_granule"), ("Q7Z739", "stress_granule"), ("O00425", "stress_granule"),
("O15234", "stress_granule"), ("P31483", "stress_granule"), ("P67809", "stress_granule"),
("Q01085", "stress_granule"), ("Q13148", "stress_granule"), ("Q13283", "stress_granule"),
("Q9BUT9", "stress_granule"), ("Q9H2U1", "stress_granule"), ("Q9NZI8", "stress_granule"),
("Q9P2E3", "stress_granule"), ("Q9UBU9", "stress_granule"), ("Q9UN86", "stress_granule"),
("Q9Y6M1", "stress_granule"),

# --- PML body ---
("P54198", "pml-body"), ("P63279", "pml-body"), ("O00178", "pml-body"),
("O94804", "pml-body"), ("P07305", "pml-body"), ("P09661", "pml-body"),
("P11171", "pml-body"), ("P18615", "pml-body"), ("P33240", "pml-body"),
("P35579", "pml-body"), ("P36578", "pml-body"), ("P41214", "pml-body"),
("Q00341", "pml-body"), ("Q12873", "pml-body"), ("Q14151", "pml-body"),
("Q14258", "pml-body"), ("Q14562", "pml-body"), ("Q15056", "pml-body"),
("Q16513", "pml-body"), ("Q69YN4", "pml-body"), ("Q7Z417", "pml-body"),
("Q86TB9", "pml-body"), ("Q8IX01", "pml-body"), ("Q8N163", "pml-body"),
("Q8N5P1", "pml-body"), ("Q8TEQ6", "pml-body"), ("Q8WW12", "pml-body"),
("Q96QC0", "pml-body"), ("Q99590", "pml-body"), ("Q9BVP2", "pml-body"),
("Q9BXJ9", "pml-body"), ("Q9C0C9", "pml-body"), ("Q9H6S0", "pml-body"),
("Q9NRA8", "pml-body"), ("Q9NWH9", "pml-body"), ("Q9UGU0", "pml-body"),
("Q9UHB7", "pml-body"), ("Q9ULX6", "pml-body"), ("Q9Y2S6", "pml-body"),

# --- Cajal body ---
("O14746", "cajal_body"), ("O14893", "cajal_body"), ("O14980", "cajal_body"),
("O94906", "cajal_body"), ("O95402", "cajal_body"), ("O95453", "cajal_body"),
("O95602", "cajal_body"), ("P06493", "cajal_body"), ("P08579", "cajal_body"),
("P08621", "cajal_body"), ("P17480", "cajal_body"), ("P35269", "cajal_body"),
("P35637", "cajal_body"), ("P38432", "cajal_body"), ("P49368", "cajal_body"),
("P61289", "cajal_body"), ("P62304", "cajal_body"), ("P62308", "cajal_body"),
("P62314", "cajal_body"), ("P62316", "cajal_body"), ("P63162", "cajal_body"),
("P63165", "cajal_body"), ("P63267", "cajal_body"), ("Q14011", "cajal_body"),
("Q14207", "cajal_body"), ("Q14331", "cajal_body"), ("Q15020", "cajal_body"),
("Q16637", "cajal_body"), ("Q5JVS0", "cajal_body"), ("Q5W0Q7", "cajal_body"),
("Q6IQ16", "cajal_body"), ("Q6NT76", "cajal_body"), ("Q7Z6G8", "cajal_body"),
("Q8IYM9", "cajal_body"), ("Q8N5A5", "cajal_body"), ("Q8NFU7", "cajal_body"),
("Q8WWY3", "cajal_body"), ("Q8WXD5", "cajal_body"), ("Q92793", "cajal_body"),
("Q96CJ1", "cajal_body"), ("Q96GM8", "cajal_body"), ("Q96HW7", "cajal_body"),
("Q96JC9", "cajal_body"), ("Q96RS0", "cajal_body"), ("Q99986", "cajal_body"),
("Q9BPY3", "cajal_body"), ("Q9BUR4", "cajal_body"), ("Q9BXT4", "cajal_body"),
("Q9GZU8", "cajal_body"), ("Q9H089", "cajal_body"), ("Q9H244", "cajal_body"),
("Q9H633", "cajal_body"), ("Q9H840", "cajal_body"), ("Q9NPE3", "cajal_body"),
("Q9NRM2", "cajal_body"), ("Q9NWZ8", "cajal_body"), ("Q9NY26", "cajal_body"),
("Q9UBY9", "cajal_body"), ("Q9UKL3", "cajal_body"), ("Q9Y230", "cajal_body"),
("Q9Y265", "cajal_body"), ("Q9Y2F5", "cajal_body"), ("Q9Y2X3", "cajal_body"),
("Q9Y3F4", "cajal_body"), ("Q9Y4X5", "cajal_body"),


]
EXTRA_IDS = []  # e.g., ["Q8NDV7"]

# ---- Stage params ---- the tiling is split into two stages 
STAGE1_TILE_LEN = 115 # length of tiles in stage-1
STAGE1_STRIDE   = 1 # step size between tiles in stage-1
STAGE2_TILE_LENGTHS = [66] # lengths of tiles in stage-2
STAGE2_STRIDE   = 1 # step size between tiles in stage-2

# ---- Thresholds ----
DISORDER_THR = 0.50
TRUTH_THR    = 0.65

# ---- Stage-1 identity dedupe knob (N→C greedy) ----
STAGE1_MAX_IDENTITY = 0.25  # drop later 115aa tiles with ≥25% identity to any earlier kept tile

# ---- Env / model paths ----
metapredict_env = "metapredict_env" # name of conda env with metapredict installed
METAPREDICT_CONDA_ENV = metapredict_env # conda env with metapredict installed

PROTGPS_PARENT_DIR = "/Users/marcellamirabelli/Desktop/rbp_localization_project/models/ProtGPS/protgps-main" # parent dir of ProtGPS repo
ESM_PRETRAIN_DIR   = "/users/marcellamirabelli/protgps/esm_models/esm2" # dir with ESM-2 pretrained models
PROTGPS_ARGS_PATH  = f"{PROTGPS_PARENT_DIR}/protgps/checkpoints/protgps/32bf44b16a4e770a674896b81dfb3729.args" # ProtGPS args path
PROTGPS_CKPT_PATH  = f"{PROTGPS_PARENT_DIR}/protgps/checkpoints/protgps/32bf44b16a4e770a674896b81dfb3729epoch=26.ckpt" # ProtGPS checkpoint path
PROTGPS_BATCH_SIZE = 8 # ProtGPS batch size (adjust based on GPU memory)

# ---- Run folder (timestamped) ----
RUN_DIR = Path(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")) # run folder (timestamped)
RUN_DIR.mkdir(parents=True, exist_ok=True) # create run folder if it doesn't exist
print("[Cell1] RUN_DIR:", RUN_DIR.resolve()) # print run folder path
print("[Cell1] Params:", #  print params for record-keeping
      
      dict(STAGE1_TILE_LEN=STAGE1_TILE_LEN, # stage-1 tile length
           STAGE1_STRIDE=STAGE1_STRIDE,# stage-1 stride
           STAGE2_TILE_LENGTHS=STAGE2_TILE_LENGTHS, # stage-2 tile lengths
           STAGE2_STRIDE=STAGE2_STRIDE,# stage-2 stride
           DISORDER_THR=DISORDER_THR,# disorder threshold
           TRUTH_THR=TRUTH_THR,# truth threshold
           STAGE1_MAX_IDENTITY=STAGE1_MAX_IDENTITY)) # stage-1 max identity


[Cell1] RUN_DIR: /Users/marcellamirabelli/Desktop/rbp_localization_project/2025-09-26_13-10-03
[Cell1] Params: {'STAGE1_TILE_LEN': 115, 'STAGE1_STRIDE': 1, 'STAGE2_TILE_LENGTHS': [66], 'STAGE2_STRIDE': 1, 'DISORDER_THR': 0.5, 'TRUTH_THR': 0.65, 'STAGE1_MAX_IDENTITY': 0.25}


Sanity Check 1:

 Checking the protein sequence that is pulled from df_master.pkl is correct by checking it with the pasted sequence from uniprot website

In [3]:
import pandas as pd # for dataframes
from collections import Counter # for counting substitutions

# --- Load df_master & pull your test protein ---
df_master = pd.read_pickle(DF_MASTER_PATH) # load df_master
uid = ID_TRUTH[0][0] # pull first test protein uniprot ID

seq_vals = df_master.loc[df_master["uniprot_id"].astype(str) == str(uid), "full_sequence"].values
if len(seq_vals) == 0:
    raise ValueError(f"[ERROR] {uid} not found in df_master")

dfm_seq = str(seq_vals[0]).replace(" ", "").replace("\n", "")
print(f"\n[SEQ from df_master for {uid}]")
print(f"length: {len(dfm_seq)}")

# --- Paste UniProt seq here --- for O94804
uniprot_seq = """
MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVGPRPPRGPLAELPSARKMNMLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPPLTPPARCAAPGGVRKAPAPFRPASERFAATTVEEILAKMEQPRKEVLASPDRLWGSRLTFNHDGSSRYGPRTYGTTTAPRDEDGSTLFRGWSQEGPVKSPAECREEHSKTPEERSLPSDLAFNGDLAKAASSELPADISKPWIPSSPAPSSENGGPASPGLPAEASGSGPGSPHLHPPDKSSPCHSQLLEAQTPEASQASPCPAVTPSAPSAALPDEGSRHTPSPGLPAEGAPEAPRPSSPPPEVLEPHSLDQPPATSPRPLIEVGELLDLTRTFPSGGEEEAKGDAHLRPTSLVQRRFSEGVLQSPSQDQEKLGGSLAALPQGQGSQLALDRPFGAESNWSLSQSFEWTFPTRPSGLGVWRLDSPPPSPITEASEAAEAAEAGNLAVSSREEGVSQQGQGAGSAPSGSGSSWVQGDDPSMSLTQKGDGESQPQFPAVPLEPLPTTEGTPGLPLQQAEERYESQEPLAGQESPLPLATREAALPILEPVLGQEQPAAPDQPCVLFADAPEPGQALPVEEEAVTLARAETTQARTEAQDLCRASPEPPGPESSSRWLDDLLASPPPSGGGARRGAGAELKDTQSPSTCSEGLLGWSQKDLQSEFGITGDPQPSSFSPSSWCQGASQDYGLGGASPRGDPGLGERDWTSKYGQGAGEGSTREWASRCGIGQEEMEASSSQDQSKVSAPGVLTAQDRVVGKPAQLGTQRSQEADVQDWEFRKRDSQGTYSSRDAELQDQEFGKRDSLGTYSSRDVSLGDWEFGKRDSLGAYASQDANEQGQDLGKRDHHGRYSSQDADEQDWEFQKRDVSLGTYGSRAAEPQEQEFGKSAWIRDYSSGGSSRTLDAQDRSFGTRPLSSGFSPEEAQQQDEEFEKKIPSVEDSLGEGSRDAGRPGERGSGGLFSPSTAHVPDGALGQRDQSSWQNSDASQEVGGHQERQQAGAQGPGSADLEDGEMGKRGWVGEFSLSVGPQREAAFSPGQQDWSRDFCIEASERSYQFGIIGNDRVSGAGFSPSSKMEGGHFVPPGKTTAGSVDWTDQLGLRNLEVSSCVGSGGSSEARESAVGQMGWSGGLSLRDMNLTGCLESGGSEEPGGIGVGEKDWTSDVNVKSKDLAEVGEGGGHSQARESGVGQTDWSGVEAGEFLKSRERGVGQADWTPDLGLRNMAPGAVCSPGESKELGVGQMDWGNNLGLRDLEVTCDPDSGGSQGLRGCGVGQMDWTQDLAPQNVELFGAPSEAREHGVGGVSQCPEPGLRHNGSLSPGLEARDPLEARELGVGETSGPETQGEDYSSSSLEPHPADPGMETGEALSFGASPGRCPARPPPSGSQGLLEEMLAASSSKAVARRESAASGLGGLLEEEGAGAGAAQEEVLEPGRDSPPSWRPQPDGEASQTEDVDGTWGSSAARWSDQGPAQTSRRPSQGPPARSPSQDFSFIEDTEILDSAMYRSRANLGRKRGHRAPVIRPGGTLGLSEAADSDAHLFQDSTEPRASRVPSSDEEVVEEPQSRRTRMSLGTKGLKVNLFPGLSPSALKAKLRPRNRSAEEGELAESKSSQKESAVQRSKSCKVPGLGKPLTLPPKPEKSSGSEGSSPNWLQALKLKKKKV
""".replace(" ", "").replace("\n", "") # paste UniProt seq here

print(f"\n[SEQ from UniProt pasted]") # print pasted UniProt seq
print(f"length: {len(uniprot_seq)}") # print length of pasted UniProt seq

# --- Compare all positions ---
min_len = min(len(dfm_seq), len(uniprot_seq)) # minimum length of the two sequences
mismatch_idxs = [i for i in range(min_len) if dfm_seq[i] != uniprot_seq[i]] # indices of mismatches within the overlapping region
extra_df  = list(range(min_len, len(dfm_seq)))      # df_master overhang (dfm longer) 
extra_uni = list(range(min_len, len(uniprot_seq)))  # UniProt overhang (uni longer)

if not mismatch_idxs and not extra_df and not extra_uni: # if no mismatches and no length differences
    print("\n✅ Sequences are an EXACT match.") # sequences match exactly
else: # if there are mismatches or length differences
    print("\n❌ Sequences differ.") # sequences differ
    print(f"  • mismatches within overlap: {len(mismatch_idxs)}") # number of mismatches within the overlapping region
    if extra_df: # if df_master sequence is longer
        print(f"  • extra df_master tail length: {len(extra_df)} (positions {min_len+1}..{len(dfm_seq)})") # print length and positions of df_master overhang
    if extra_uni: # if UniProt sequence is longer
        print(f"  • extra UniProt tail length : {len(extra_uni)} (positions {min_len+1}..{len(uniprot_seq)})") # print length and positions of UniProt overhang

    # --- Group mismatches into contiguous runs ---
    def group_runs(idxs): # group indices into contiguous runs
        runs = [] # list to hold runs
        for i in idxs:
            if not runs or i != runs[-1][1] + 1: # if runs is empty or current index is not contiguous with last run
                runs.append([i, i]) #   start a new run
            else:       
                runs[-1][1] = i    #   extend the last run
        return runs

    runs = group_runs(mismatch_idxs)

    # --- Show each run with a little context ---
    CONTEXT = 5  # aa to show before/after
    for start, end in runs:
        # 1-based display indices
        disp_start, disp_end = start + 1, end + 1   # convert to 1-based indexing for display
        left = max(0, start - CONTEXT) # left context
        right = min(min_len, end + 1 + CONTEXT) # right context

        df_slice  = dfm_seq[left:right] # slice from df_master sequence
        uni_slice = uniprot_seq[left:right] # slice from UniProt sequence

        # caret marker for the mismatched span
        caret_pad = " " * (start - left) # padding before carets
        carets = "^" * (end - start + 1) # carets for mismatched positions

        print(f"\n[Mismatch run] positions {disp_start}..{disp_end} (len={end-start+1})") # print mismatch run info
        print(f"df_master[{left+1}:{right}] : {df_slice}") # print df_master slice
        print(f"UniProt  [{left+1}:{right}] : {uni_slice}") # print UniProt slice
        print(f"               {caret_pad}{carets}") # print carets under mismatched positions

    # --- Quick substitution summary (within the overlapping region) ---
    sub_counts = Counter((dfm_seq[i], uniprot_seq[i]) for i in mismatch_idxs) # count substitutions
    if sub_counts: # if there are any substitutions
        print("\n[Substitution summary within overlap] (df_master → UniProt)  count") # print header
        for (a,b), c in sub_counts.most_common(): # for each substitution, sorted by count
            print(f"  {a} → {b} : {c}") # print substitution and count

    # --- If there are length differences, show the overhang strings too ---
    if extra_df: # if df_master sequence is longer
        print("\n[df_master overhang]") # print header
        print(dfm_seq[min_len:]) # print df_master overhang
    if extra_uni: # if UniProt sequence is longer
        print("\n[UniProt overhang]") # print header
        print(uniprot_seq[min_len:]) # print UniProt overhang



[SEQ from df_master for Q9C0C2]
length: 1729

[SEQ from UniProt pasted]
length: 1729

✅ Sequences are an EXACT match.


Cell 2:

Goal of this cell: to normalize the labels of the truth labels so it matches what I have in my dataframe, checks for the synonyms or hyphins that may come up so we have one label for everything. 

Load and validate the sequences I will tile: reports any IDs that were not found. 

In [4]:
# ===== Cell 2: Canonical labels + load df_master =====
import pandas as pd

def _norm(s): #_norm normalizes a string by lowercasing and removing non-alphanumerics.
    return "".join(ch for ch in str(s).lower() if ch.isalnum()) # keep only alphanumerics and lowercase

_SYNONYMS = { #_SYNONYMS maps many spellings → one canonical label.
    "pbody": "p-body", "pbodies": "p-body",
    "pmlbody": "pml-body", "pml": "pml-body", "pml-bdoy": "pml-body",
    "stressgranule": "stress_granule",
    "nuclearspeckle": "nuclear_speckle",
    "cajalbody": "cajal_body", "cajal body": "cajal_body",
}
ALLOWED_TRUTHS = {"nuclear_speckle","p-body","pml-body","stress_granule","nucleolus","cajal_body","chromosome"} #_ALLOWED_TRUTHS is the list of labels you’ll accept downstream.

def canonicalize_label(label): #canonicalize_label maps a label to its canonical form, or returns None if the label is None.
    if label is None: # if label is None
        return None # return None
    return _SYNONYMS.get(_norm(label), label) # return canonical label or original label if not found

def dedupe_preserve_order(items): # dedupe_preserve_order removes duplicates while preserving order.
    seen, out = set(), [] # seen tracks seen items, out is the output list
    for x in items: # iterate over input items
        if x not in seen: # if item is not seen
            out.append(x); seen.add(x) # add to output and mark as seen
    return out

UNIPROT_IDS = dedupe_preserve_order([str(uid) for uid, _ in ID_TRUTH] + list(EXTRA_IDS)) # all unique uniprot IDs in order
TRUTH_MAP   = {uid: canonicalize_label(truth) for (uid, truth) in ID_TRUTH if truth} # map of uniprot ID to canonical truth label

df_master = pd.read_pickle(DF_MASTER_PATH) # load df_master
need_cols = {"uniprot_id", "full_sequence"} #Ensures your master table has the expected columns.
if not need_cols.issubset(df_master.columns): # if required columns are missing
    raise ValueError(f"df_master must have {need_cols}. Got: {list(df_master.columns)}") # raise error if columns are missing

seqmap = dict(zip(df_master["uniprot_id"].astype(str), # map of uniprot_id to full_sequence
                  df_master["full_sequence"].astype(str))) 

present = {uid: seqmap.get(uid,"") # present IDs with non-empty sequences
           for uid in UNIPROT_IDS # in order of UNIPROT_IDS
           if isinstance(seqmap.get(uid,""), str) and len(seqmap.get(uid,""))>0}
missing_ids = [uid for uid in UNIPROT_IDS if uid not in present]

print(f"[Cell2] Found {len(present)} / {len(UNIPROT_IDS)} IDs. Missing: {len(missing_ids)}")
if missing_ids: print("[Cell2] Missing IDs:", missing_ids)


[Cell2] Found 120 / 168 IDs. Missing: 48
[Cell2] Missing IDs: ['P54198', 'O14746', 'O14893', 'O14980', 'O95402', 'O95602', 'P06493', 'P08579', 'P38432', 'P61289', 'P62304', 'P63162', 'P63267', 'Q14207', 'Q16637', 'Q5W0Q7', 'Q6IQ16', 'Q6NT76', 'Q7Z6G8', 'Q8IYM9', 'Q8N5A5', 'Q8NFU7', 'Q8WXD5', 'Q92793', 'Q96CJ1', 'Q96GM8', 'Q96HW7', 'Q96JC9', 'Q96RS0', 'Q99986', 'Q9BPY3', 'Q9BUR4', 'Q9BXT4', 'Q9GZU8', 'Q9H089', 'Q9H244', 'Q9H633', 'Q9H840', 'Q9NPE3', 'Q9NRM2', 'Q9NWZ8', 'Q9NY26', 'Q9UBY9', 'Q9UKL3', 'Q9Y230', 'Q9Y265', 'Q9Y2F5', 'Q9Y4X5']


Sanity Check 2: 

Just checking the columns and structure of the dataframe so that it is getting the correct information downstream.

In [5]:
import pandas as pd # for dataframes

# Load the master dataframe
df_master = pd.read_pickle(DF_MASTER_PATH) 

# ---- Column sanity checks ----
need_cols = {"uniprot_id", "full_sequence"}
have_cols = set(df_master.columns)

missing = need_cols - have_cols
extra   = have_cols - need_cols

if missing:
    raise ValueError(f"[df_master] Missing required columns: {missing}")
else:
    print(f"[df_master] ✓ All required columns present: {need_cols}")

if extra:
    print(f"[df_master] (info) Extra columns found: {extra}")

# ---- Enforce dtypes and cleaning ----
df_master["uniprot_id"]    = df_master["uniprot_id"].astype(str).str.strip()
df_master["full_sequence"] = (
    df_master["full_sequence"]
    .astype(str)
    .str.replace(r"\s+", "", regex=True)  # remove whitespace/newlines
)

print(f"[df_master] Shape: {df_master.shape}")
print(f"[df_master] Example rows:\n{df_master.head(3)}")


[df_master] ✓ All required columns present: {'full_sequence', 'uniprot_id'}
[df_master] (info) Extra columns found: {'num_disordered_regions', 'disordered_seqs', 'localization_hpa', 'localization_uniprot', 'gene_name', 'has_disorder'}
[df_master] Shape: (1380, 8)
[df_master] Example rows:
  gene_name uniprot_id                                      full_sequence  \
0      A1CF     Q9NQ94  MESNHKSGDGLSGTQKEAALRALVQRTGYSLVQENGQRKYGGPPPG...   
1      AARS     P49588  MDSTLTASEIRQRFIDFFKRNEHTYVHSSATIPLDDPTLLFANAGM...   
2      AATF     Q9NY61  MAGPQPLALQLEQLLNPRPSEADPEADPEEATAARVIDRFDEGEDG...   

                                     disordered_seqs  num_disordered_regions  \
0                                                 []                       0   
1                                                 []                       0   
2  [TSRKAWNEDHWEQTLPGSSDEEISDEEGSGDEDSEGLGLEEYDED...                       1   

   has_disorder      localization_uniprot      localization_hpa  
0         Fals

Inspecting the row from the test protein I am looking at 

In [6]:
# === Inspect the full row for the test UniProt ID ===
uid = ID_TRUTH[0][0]  # e.g., "Q13427"

row = df_master.loc[df_master["uniprot_id"] == uid]

if row.empty:
    print(f"[df_master] UniProt ID {uid} not found!")
else:
    print(f"\n[df_master] Full record for {uid}:")
    # pretty-print all columns with their values
    for col in row.columns:
        val = row.iloc[0][col]
        # truncate very long strings for readability
        if isinstance(val, str) and len(val) > 200:
            val = val[:100] + " ... " + val[-100:]
        print(f"  {col}: {val}")



[df_master] Full record for Q9C0C2:
  gene_name: TNKS1BP1
  uniprot_id: Q9C0C2
  full_sequence: MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVGPRPPRGPLAELPSARKMNMLAGPQPYGGSKRPLPFA ... QSRRTRMSLGTKGLKVNLFPGLSPSALKAKLRPRNRSAEEGELAESKSSQKESAVQRSKSCKVPGLGKPLTLPPKPEKSSGSEGSSPNWLQALKLKKKKV
  disordered_seqs: ['MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVGPRPPRGPLAELPSARKMNMLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPPLTPPARCAAPGGVRKAPAPFRP', 'GSRLTFNHDGSSRYGPRTYGTTTAPRDEDGSTLFRGWSQEGPVKSPAECREEHSKTPEERSLPSDLAFNGDLAKAASSELPADISKPWIPSSPAPSSENGGPASPGLPAEASGSGPGSPHLHPPDKSSPCHSQLLEAQTPEASQASPCPAVTPSAPSAALPDEGSRHTPSPGLPAEGAPEAPRPSSPPPEVLEPHSLDQPPATSPRPLIEVGELLDLTRTFPSGGEEEAKGDAHLRPTSLVQRRFSEGVLQSPSQDQEKLGGSLAAL', 'PSGLGVWRLDSPPPSPITEASEAAEAAEAGNLAVSSREEGVSQQGQGAGSAPSGSGSSWVQGDDPSMSLTQKGDGESQPQFPAVPLEPLPTTEGTPGLPLQQAEERYESQEPLAGQESPL', 'ETTQARTEAQDLCRASPEPPGPESSSRWLDDLLASPPPSGGGARRGAGAELKDTQSPSTCSEGLLGWSQKDLQSEFGITGDPQPSSFSPSSWCQGASQDYGLGGASPRGDPGLGERDWTSKYGQGA

Cell 3: 

Building the 115aa tiles by a 1aa window 

Number of tiles is calculated by: # amino acids - total length of tile + 1 = # of tiles generated 


This example: 754aa -115 +1 = 640 tiles made 

In [7]:
# ===== Cell 3: Build 115-aa tiles (stride=1) =====
def tile_sequence(seq, tile_len, stride=1, phase=0):
    n = len(seq) # length of sequence
    if tile_len > n: #if the tile length is greater than the sequence length, no tiles can be formed
        return [] # return an empty list
    return [(i, seq[i:i+tile_len]) for i in range(phase, n - tile_len + 1, stride)] # return list of (start_index, tile_sequence) tuples

def build_stage1_tiles(seqmap, uids, tile_len=115, stride=1): #builds a DataFrame of tiles for given UniProt IDs.
    rows = [] # list to hold tile information
    for uid in uids: # iterate over each UniProt ID
        seq = seqmap.get(uid, "") # get the sequence for the current UniProt ID
        if not isinstance(seq, str) or len(seq) < tile_len: # if sequence is not a string or is shorter than tile length
            continue # skip to the next ID
        for start, tseq in tile_sequence(seq, tile_len=tile_len, stride=stride, phase=0): # generate tiles
            rows.append({ # append tile info to rows list
                "uniprot_id": uid, # UniProt ID
                "tile_len":   tile_len, # length of the tile
                "stride":     stride, # stride used for tiling
                "phase":      0, # phase used for tiling
                "start":      start, # start index of the tile
                "end":        start + tile_len, # end index of the tile
                "tile_seq":   tseq, # the tile sequence
                "TRUTH_Label": TRUTH_MAP.get(uid) # truth label if available
            })
    return pd.DataFrame(rows) # convert rows list to DataFrame and return

S1_BASE = RUN_DIR / f"stage1_len{STAGE1_TILE_LEN}_stride{STAGE1_STRIDE}" # base path for stage-1 tiles
s1_tiles = build_stage1_tiles(seqmap, list(present.keys()), STAGE1_TILE_LEN, STAGE1_STRIDE) # build stage-1 tiles
s1_tiles_path = S1_BASE.with_suffix(".tiles.csv") # path to save stage-1 tiles
s1_tiles.to_csv(s1_tiles_path, index=False) # save stage-1 tiles to CSV

print(f"[Cell3] Stage-1 tiles: {len(s1_tiles):,}") # print number of stage-1 tiles
print(f"[Cell3] Wrote: {s1_tiles_path.name}") # print name of the saved file
display(s1_tiles.head(3)) # display first 3 rows of stage-1 tiles


[Cell3] Stage-1 tiles: 82,576
[Cell3] Wrote: stage1_len115_stride1.tiles.csv


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label
0,Q9C0C2,115,1,0,0,115,MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRA...,chromosome
1,Q9C0C2,115,1,0,1,116,KVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRAL...,chromosome
2,Q9C0C2,115,1,0,2,117,VSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALP...,chromosome


Sanity check 4: 


Double checking we get the same number of tiles as calculated 

In [8]:
### just checking how many tiles should be in theory made 
# ==== Stage-1 tiling sanity checks (stride=1, phase=0, L=115) ====
from collections import defaultdict

def expected_tiles(n, L=STAGE1_TILE_LEN, stride=STAGE1_STRIDE, phase=0): # calculate expected number of tiles
    if n < L: 
        return 0
    # general formula; reduces to n-L+1 for stride=1, phase=0
    return ((n - L - phase) // stride) + 1 # +1 to include the last tile starting at n-L

problems = [] #list of (uid, kind, info) tuples describing issues found
by_uid_stats = [] #list of (uid, n, expected, got) tuples

for uid in present.keys():
    seq = seqmap.get(uid, "")
    n = len(seq) if isinstance(seq, str) else 0
    exp = expected_tiles(n, L=STAGE1_TILE_LEN, stride=STAGE1_STRIDE, phase=0)

    sub = s1_tiles[s1_tiles["uniprot_id"] == uid].copy()
    got = len(sub)

    # record stats
    by_uid_stats.append((uid, n, exp, got))

    # quick shape checks (only if there are any tiles)
    if got:
        # start/end boundary checks
        first_start = int(sub["start"].min())
        last_start  = int(sub["start"].max())
        first_end   = int(sub.loc[sub["start"].idxmin(), "end"])
        last_end    = int(sub.loc[sub["start"].idxmax(), "end"])

        # Assert consistent tile length
        bad_len = sub[(sub["end"] - sub["start"]) != STAGE1_TILE_LEN]
        if not bad_len.empty:
            problems.append((uid, "tile_len_mismatch", bad_len.index.tolist()))

        # Assert boundaries
        if first_start != 0:
            problems.append((uid, "first_start_not_zero", first_start))
        if last_start != n - STAGE1_TILE_LEN:
            problems.append((uid, "last_start_wrong", (last_start, n - STAGE1_TILE_LEN)))
        if first_end != STAGE1_TILE_LEN:
            problems.append((uid, "first_end_wrong", first_end))
        if last_end != n:
            problems.append((uid, "last_end_wrong", (last_end, n)))

    # Count mismatch?
    if got != exp:
        problems.append((uid, "count_mismatch", (exp, got)))

# ---- Pretty print summary ----
print("\n[Stage-1 tile math check]")
for uid, n, exp, got in by_uid_stats:
    print(f"  {uid}: length={n} | expected_tiles={exp} | got={got}")

if problems:
    print("\n[WARN] Detected issues:")
    for uid, kind, info in problems:
        print(f"  - {uid}: {kind}: {info}")
else:
    print("\n✓ All stage-1 tiling counts and boundaries look correct.")

# Show head/tail of this protein’s tiles to eyeball start/end
uid0 = next(iter(present.keys()))
sub0 = s1_tiles[s1_tiles["uniprot_id"] == uid0].sort_values("start")
print(f"\nExample ({uid0}) first/last tiles:")
print(sub0.head(2)[["start","end","tile_len","tile_seq"]])
print(sub0.tail(2)[["start","end","tile_len","tile_seq"]])




[Stage-1 tile math check]
  Q9C0C2: length=1729 | expected_tiles=1615 | got=1615
  Q8NEY8: length=458 | expected_tiles=344 | got=344
  P46013: length=3256 | expected_tiles=3142 | got=3142
  Q96SB4: length=655 | expected_tiles=541 | got=541
  Q14684: length=758 | expected_tiles=644 | got=644
  Q5UIP0: length=2472 | expected_tiles=2358 | got=2358
  Q13427: length=754 | expected_tiles=640 | got=640
  Q7L4I2: length=434 | expected_tiles=320 | got=320
  Q9Y383: length=392 | expected_tiles=278 | got=278
  O75400: length=957 | expected_tiles=843 | got=843
  O95232: length=432 | expected_tiles=318 | got=318
  P18583: length=2426 | expected_tiles=2312 | got=2312
  Q01130: length=221 | expected_tiles=107 | got=107
  Q02040: length=695 | expected_tiles=581 | got=581
  Q13435: length=895 | expected_tiles=781 | got=781
  Q14004: length=1512 | expected_tiles=1398 | got=1398
  Q14498: length=530 | expected_tiles=416 | got=416
  Q16630: length=551 | expected_tiles=437 | got=437
  Q6PJT7: length=736 |

Cell 4: 

Runs metapredict in another environment and uses the input from the first tiling csv made 

In [9]:
# ===== Cell 4: Run MetaPredict on Stage-1 tiles =====
import os, shutil, subprocess #for running MetaPredict in a separate conda env

def run_metapredict(input_csv: Path, out_prefix: Path, mean_thr: float): #runs MetaPredict on input CSV of tiles, filters by mean disorder threshold, and saves results.
    inline_code = r'''
import pandas as pd, numpy as np, sys
try:
    import metapredict as mp
except Exception as e:
    raise SystemExit("ERROR: metapredict not available in this environment") from e
inp, out_res, out_kept, thr = sys.argv[1], sys.argv[2], sys.argv[3], float(sys.argv[4])
df = pd.read_csv(inp)
need = {"uniprot_id","tile_len","stride","phase","start","end","tile_seq"}
missing = need - set(df.columns)
if missing:
    raise SystemExit(f"Input CSV missing columns: {missing}")
def avg_disorder(seq: str) -> float:
    if not isinstance(seq, str) or not seq: return 0.0
    scores = np.asarray(mp.predict_disorder(seq), dtype=float)
    return float(scores.mean()) if scores.size else 0.0
try:
    from tqdm.auto import tqdm
    tqdm.pandas(desc="MetaPredict (mean disorder)")
    df["disorder_mean"] = df["tile_seq"].progress_apply(avg_disorder)
except Exception:
    df["disorder_mean"] = [avg_disorder(s) for s in df["tile_seq"]]
df.to_csv(out_res, index=False)
kept = df[df["disorder_mean"] > thr].copy()
kept.to_csv(out_kept, index=False)
print(f"scored={len(df)} kept={len(kept)}")
'''
    launcher = os.environ.get("CONDA_EXE") or shutil.which("conda") or shutil.which("mamba") or shutil.which("micromamba")
    if not launcher:
        raise RuntimeError("Conda/mamba/micromamba not found in PATH or CONDA_EXE.")
    out_res  = out_prefix.with_suffix("").as_posix() + "_metapredict_RESULTS.csv"
    out_kept = out_prefix.with_suffix("").as_posix() + f"_metapredict_KEPT_mean{str(mean_thr).replace('.','p')}.csv"
    cmd = [launcher, "run", "-n", METAPREDICT_CONDA_ENV, "python", "-c", inline_code,
           str(input_csv), out_res, out_kept, str(mean_thr)]
    print("[Cell4][MetaPredict CMD]", " ".join(cmd))
    res = subprocess.run(cmd, capture_output=True, text=True)
    print("[Cell4][MetaPredict STDOUT]\n", (res.stdout or "").strip())
    if res.returncode != 0:
        print("[Cell4][MetaPredict STDERR]\n", (res.stderr or "").strip())
        raise RuntimeError("MetaPredict failed.")
    return Path(out_res), Path(out_kept)

s1_results_path, s1_kept_path = run_metapredict(s1_tiles_path, S1_BASE, DISORDER_THR)
print(f"[Cell4] Wrote: {s1_results_path.name}")
print(f"[Cell4] Wrote: {s1_kept_path.name}")


[Cell4][MetaPredict CMD] /opt/anaconda3/bin/conda run -n metapredict_env python -c 
import pandas as pd, numpy as np, sys
try:
    import metapredict as mp
except Exception as e:
    raise SystemExit("ERROR: metapredict not available in this environment") from e
inp, out_res, out_kept, thr = sys.argv[1], sys.argv[2], sys.argv[3], float(sys.argv[4])
df = pd.read_csv(inp)
need = {"uniprot_id","tile_len","stride","phase","start","end","tile_seq"}
missing = need - set(df.columns)
if missing:
    raise SystemExit(f"Input CSV missing columns: {missing}")
def avg_disorder(seq: str) -> float:
    if not isinstance(seq, str) or not seq: return 0.0
    scores = np.asarray(mp.predict_disorder(seq), dtype=float)
    return float(scores.mean()) if scores.size else 0.0
try:
    from tqdm.auto import tqdm
    tqdm.pandas(desc="MetaPredict (mean disorder)")
    df["disorder_mean"] = df["tile_seq"].progress_apply(avg_disorder)
except Exception:
    df["disorder_mean"] = [avg_disorder(s) for s in df["ti

SANITY CHECK 5

Running metapredict from the base code and pasting in the first tile of the library made and seeing if I get the same average of disorder for that tile

checked average in this file /Users/marcellamirabelli/Desktop/rbp_localization_project/Metapredict_Tiling_Sanity_Check.ipynb

In [10]:
import pandas as pd
# also ran tile in /Users/marcellamirabelli/Desktop/rbp_localization_project/Metapredict_Tiling_Sanity_Check.ipynb
# load the MetaPredict results
df_res = pd.read_csv(s1_results_path) # load the MetaPredict results

uid = ID_TRUTH[0][0]  # your test protein ID, e.g. "Q13427"

# grab the first tile (start=0)
first_tile_row = df_res.query("uniprot_id == @uid & start == 0").iloc[0] # get the first tile row

tile_seq = first_tile_row["tile_seq"] # extract the tile sequence
avg_disorder = first_tile_row["disorder_mean"] # extract the average disorder score

print(f"[First tile for {uid}]") # print header
print(f"start={first_tile_row['start']} end={first_tile_row['end']}") # print start and end positions
print(f"length={len(tile_seq)} aa") # print length of the tile
print(f"avg disorder={avg_disorder:.3f}") # print average disorder score
print(f"tile sequence:\n{tile_seq}") # print the tile sequence


[First tile for Q9C0C2]
start=0 end=115
length=115 aa
avg disorder=0.903
tile sequence:
MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVGPRPPRGPLAELPSARKMNMLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQ


Sanity check 6: 

Before running prot GPS, we are running the github test sequences first and seeing if we get the same results: https://github.com/pgmikhael/protgps/blob/main/notebook/Predict.ipynb

Reference output for the control protein sequences:

O15116

NUCLEAR_SPECKLE_Score 0.0
P-BODY_Score 0.9990000128746033
PML-BDOY_Score 0.0
POST_SYNAPTIC_DENSITY_Score 0.0
STRESS_GRANULE_Score 0.0
CHROMOSOME_Score 0.0
NUCLEOLUS_Score 0.0010000000474974513
NUCLEAR_PORE_COMPLEX_Score 0.0010000000474974513
CAJAL_BODY_Score 0.0
RNA_GRANULE_Score 0.0
CELL_JUNCTION_Score 0.0
TRANSCRIPTIONAL_Score 0.0
P38432

NUCLEAR_SPECKLE_Score 0.0
P-BODY_Score 0.0
PML-BDOY_Score 0.003000000026077032
POST_SYNAPTIC_DENSITY_Score 0.0
STRESS_GRANULE_Score 0.0
CHROMOSOME_Score 0.0
NUCLEOLUS_Score 0.004000000189989805
NUCLEAR_PORE_COMPLEX_Score 0.0020000000949949026
CAJAL_BODY_Score 0.9919999837875366
RNA_GRANULE_Score 0.0
CELL_JUNCTION_Score 0.0
TRANSCRIPTIONAL_Score 0.0

In [11]:
# ===== PRE-CELL: ProtGPS controls (NO tiling, NO disorder) =====
# Purpose: run ProtGPS once on a couple of whole control sequences to confirm the model pipeline is correct.

import numpy as np
import pandas as pd
import torch
import pickle, sys
from argparse import Namespace
from pathlib import Path

# --- If these aren't already defined in your session, define them defensively ---
try:
    COMPARTMENT_CLASSES
except NameError:
    COMPARTMENT_CLASSES = [
        "nuclear_speckle",
        "p-body",
        "pml-body",
        "post_synaptic_density",
        "stress_granule",
        "chromosome",
        "nucleolus",
        "nuclear_pore_complex",
        "cajal_body",
        "rna_granule",
        "cell_junction",
        "transcriptional",
    ]

def _score_col(lbl: str) -> str:
    return f"{str(lbl).upper().replace(' ','').replace('-','')}_Score"

try:
    predict_condensates
except NameError:
    @torch.no_grad()
    def predict_condensates(model, sequences, batch_size=1, round=False):
        scores = []
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i+batch_size]
            out = model.model({"x": batch})
            s = torch.sigmoid(out['logit']).to("cpu")
            scores.append(s)
        scores = torch.vstack(scores)
        return torch.round(scores, decimals=3) if round else scores

# --- Controls (whole sequences; use exactly as-given, no tiling) ---
controls = {
    "CTRL_O15116":
        "MNYMPGTASLIEDIDKKHLVLLRDGRTLIGFLRSIDQFANLVLHQTVERIHVGKKYGDIPRGIFVVRGENVVLLGEIDLEKESDTPLQQVSIEEILEEQRVEQQTKLEAEKLKVQALKDRGLSIPRADTLDEY",
    "CTRL_P38432":
        "MAASETVRLRLQFDYPPPATPHCTAFWLLVDLNRCRVVTDLISLIRQRFGFSSGAFLGLYLEGGLLPPAESARLVRDNDCLRVKLEERGVAENSVVISNGDINLSLRKAKKRAFQLEEGEETEPDCKYSKKHWKSRENNNNNEKVLDLEPKAVTDQTVSKKNKRKNKATCGTVGDDNEEAKRKSPKKKEKCEYKKKAKNPKSPKVQAVKDWANQRCSSPKGSARNSLVKAKRKGSVSVCSKESPSSSSESESCDESISDGPSKVTLEARNSSEKLPTELSKEEPSTKNTTADKLAIKLGFSLTPSKGKTSGTTSSSSDSSAESDDQCLMSSSTPECAAGFLKTVGLFAGRGRPGPGLSSQTAGAAGWRRSGSNGGGQAPGASPSVSLPASLGRGWGREENLFSWKGAKGRGMRGRGRGRGHPVSCVVNRSTDNQRQQQLNDVVKNSSTIIQNPVETPKKDYSLLPLLAAAPQVGEKIAFKLLELTSSYSPDVSDYKEGRILSHNPETQQVDIEILSSLPALREPGKFDLVYHNENGAEVVEYAVTQESKITVFWKELIDPRLIIESPSNTSSTEPA"
}

# --- Load model once if not already loaded ---
try:
    model
except NameError:
    sys.path.append(PROTGPS_PARENT_DIR)
    from protgps.utils.loading import get_object
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    args = Namespace(**pickle.load(open(PROTGPS_ARGS_PATH,'rb')))
    args.model_path = PROTGPS_CKPT_PATH
    args.pretrained_hub_dir = ESM_PRETRAIN_DIR

    def load_model(snargs):
        model_ = get_object(snargs.lightning_name, "lightning")(snargs)
        model_ = model_.load_from_checkpoint(
            checkpoint_path = snargs.model_path,
            strict=not snargs.relax_checkpoint_matching,
            **{"args": snargs},
        )
        return model_

    model = load_model(args).eval().to(device)
    print("[Controls] ProtGPS model loaded on", device)

# --- Score controls (whole sequences) ---
ctrl_ids = list(controls.keys())
ctrl_seqs = [controls[k] for k in ctrl_ids]

scores_t = predict_condensates(model, sequences=ctrl_seqs, batch_size=PROTGPS_BATCH_SIZE if 'PROTGPS_BATCH_SIZE' in globals() else 8, round=True)
scores = scores_t.cpu().numpy() if hasattr(scores_t, "cpu") else np.asarray(scores_t)

# Build a nice table
df_ctrl = pd.DataFrame(scores, columns=[_score_col(c) for c in COMPARTMENT_CLASSES])
df_ctrl.insert(0, "control_id", ctrl_ids)
df_ctrl.insert(1, "length", [len(s) for s in ctrl_seqs])

# Print controls first (top-3 classes per sequence)
print("\n[Controls] ProtGPS probabilities (rounded to 3 d.p.)")
with pd.option_context('display.max_columns', None, 'display.width', 160):
    print(df_ctrl.to_string(index=False))

print("\n[Controls] Top-3 classes per sequence:")
for i, cid in enumerate(ctrl_ids):
    row = df_ctrl.iloc[i]
    top3 = row[[c for c in df_ctrl.columns if c.endswith('_Score')]].sort_values(ascending=False).head(3)
    tops = ", ".join([f"{k.replace('_Score','')}: {v:.3f}" for k, v in top3.items()])
    print(f"  - {cid} (len={row['length']}): {tops}")

# (Optional) Save controls to a small CSV for record
controls_out = Path(RUN_DIR) / "controls_PROTGPS.csv"
df_ctrl.to_csv(controls_out, index=False)
print(f"\n[Controls] Wrote: {controls_out.name}")


Using cache found in /users/marcellamirabelli/protgps/esm_models/esm2/facebookresearch_esm_main


Using ESM hidden layers 6
Using ESM hidden layers 6
[Controls] ProtGPS model loaded on cpu


Using cache found in /users/marcellamirabelli/protgps/esm_models/esm2/facebookresearch_esm_main



[Controls] ProtGPS probabilities (rounded to 3 d.p.)
 control_id  length  NUCLEAR_SPECKLE_Score  PBODY_Score  PMLBODY_Score  POST_SYNAPTIC_DENSITY_Score  STRESS_GRANULE_Score  CHROMOSOME_Score  NUCLEOLUS_Score  NUCLEAR_PORE_COMPLEX_Score  CAJAL_BODY_Score  RNA_GRANULE_Score  CELL_JUNCTION_Score  TRANSCRIPTIONAL_Score
CTRL_O15116     133                    0.0          1.0          0.000                          0.0                   0.0               0.0            0.000                       0.000             0.000                0.0                  0.0                    0.0
CTRL_P38432     576                    0.0          0.0          0.003                          0.0                   0.0               0.0            0.004                       0.002             0.992                0.0                  0.0                    0.0

[Controls] Top-3 classes per sequence:
  - CTRL_O15116 (len=133): PBODY: 1.000, NUCLEAR_SPECKLE: 0.000, PMLBODY: 0.000
  - CTRL_P38432 (len=576): C

[First tile for Q9C0C2]
start=0 end=115
length=115 aa
avg disorder=0.903
tile sequence:
MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVGPRPPRGPLAELPSARKMNMLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQ

In [12]:
## Prot GPS sanity check with original code on tile #1 
# ===== PRE-CELL: ProtGPS controls (NO tiling, NO disorder) =====
# Purpose: run ProtGPS once on a couple of whole control sequences to confirm the model pipeline is correct.

import numpy as np
import pandas as pd
import torch
import pickle, sys
from argparse import Namespace
from pathlib import Path

# --- If these aren't already defined in your session, define them defensively ---
try:
    COMPARTMENT_CLASSES
except NameError:
    COMPARTMENT_CLASSES = [
        "nuclear_speckle",
        "p-body",
        "pml-body",
        "post_synaptic_density",
        "stress_granule",
        "chromosome",
        "nucleolus",
        "nuclear_pore_complex",
        "cajal_body",
        "rna_granule",
        "cell_junction",
        "transcriptional",
    ]

def _score_col(lbl: str) -> str:
    return f"{str(lbl).upper().replace(' ','').replace('-','')}_Score"

try:
    predict_condensates
except NameError:
    @torch.no_grad()
    def predict_condensates(model, sequences, batch_size=1, round=False):
        scores = []
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i+batch_size]
            out = model.model({"x": batch})
            s = torch.sigmoid(out['logit']).to("cpu")
            scores.append(s)
        scores = torch.vstack(scores)
        return torch.round(scores, decimals=3) if round else scores

# --- Controls (whole sequences; use exactly as-given, no tiling) ---
controls = {
    "CTRL_2":
        "MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVGPRPPRGPLAELPSARKMNMLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQ",
  
}

# --- Load model once if not already loaded ---
try:
    model
except NameError:
    sys.path.append(PROTGPS_PARENT_DIR)
    from protgps.utils.loading import get_object
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    args = Namespace(**pickle.load(open(PROTGPS_ARGS_PATH,'rb')))
    args.model_path = PROTGPS_CKPT_PATH
    args.pretrained_hub_dir = ESM_PRETRAIN_DIR

    def load_model(snargs):
        model_ = get_object(snargs.lightning_name, "lightning")(snargs)
        model_ = model_.load_from_checkpoint(
            checkpoint_path = snargs.model_path,
            strict=not snargs.relax_checkpoint_matching,
            **{"args": snargs},
        )
        return model_

    model = load_model(args).eval().to(device)
    print("[Controls] ProtGPS model loaded on", device)

# --- Score controls (whole sequences) ---
ctrl_ids = list(controls.keys())
ctrl_seqs = [controls[k] for k in ctrl_ids]

scores_t = predict_condensates(model, sequences=ctrl_seqs, batch_size=PROTGPS_BATCH_SIZE if 'PROTGPS_BATCH_SIZE' in globals() else 8, round=True)
scores = scores_t.cpu().numpy() if hasattr(scores_t, "cpu") else np.asarray(scores_t)

# Build a nice table
df_ctrl = pd.DataFrame(scores, columns=[_score_col(c) for c in COMPARTMENT_CLASSES])
df_ctrl.insert(0, "control_id", ctrl_ids)
df_ctrl.insert(1, "length", [len(s) for s in ctrl_seqs])

# Print controls first (top-3 classes per sequence)
print("\n[Controls] ProtGPS probabilities (rounded to 3 d.p.)")
with pd.option_context('display.max_columns', None, 'display.width', 160):
    print(df_ctrl.to_string(index=False))

print("\n[Controls] Top-3 classes per sequence:")
for i, cid in enumerate(ctrl_ids):
    row = df_ctrl.iloc[i]
    top3 = row[[c for c in df_ctrl.columns if c.endswith('_Score')]].sort_values(ascending=False).head(3)
    tops = ", ".join([f"{k.replace('_Score','')}: {v:.3f}" for k, v in top3.items()])
    print(f"  - {cid} (len={row['length']}): {tops}")

# (Optional) Save controls to a small CSV for record
controls_out = Path(RUN_DIR) / "controls_PROTGPS.csv"
df_ctrl.to_csv(controls_out, index=False)
print(f"\n[Controls] Wrote: {controls_out.name}")



[Controls] ProtGPS probabilities (rounded to 3 d.p.)
control_id  length  NUCLEAR_SPECKLE_Score  PBODY_Score  PMLBODY_Score  POST_SYNAPTIC_DENSITY_Score  STRESS_GRANULE_Score  CHROMOSOME_Score  NUCLEOLUS_Score  NUCLEAR_PORE_COMPLEX_Score  CAJAL_BODY_Score  RNA_GRANULE_Score  CELL_JUNCTION_Score  TRANSCRIPTIONAL_Score
    CTRL_2     115                    0.0        0.974            0.0                          0.0                   0.0               0.0            0.164                         0.0               0.0                0.0                  0.0                    0.0

[Controls] Top-3 classes per sequence:
  - CTRL_2 (len=115): PBODY: 0.974, NUCLEOLUS: 0.164, NUCLEAR_SPECKLE: 0.000

[Controls] Wrote: controls_PROTGPS.csv


In [13]:
# ===== Cell 5a: ProtGPS scoring for Stage-1 (115aa) tiles =====
# loads the protGPS model
#takes the dataframes from metapredict-kept 115aa tiles from cell 4
# add a per "tile truth score" by picking the column that matches the "truth label"
# saves a new csv of these tiles with the scores added 

import sys, pickle, torch, pandas as pd, numpy as np
from argparse import Namespace
from pathlib import Path
sys.path.append(PROTGPS_PARENT_DIR)
from protgps.utils.loading import get_object

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Define ProtGPS compartment classes (match model training order)
COMPARTMENT_CLASSES = [
    "nuclear_speckle",
    "p-body",
    "pml-body",  # corrected spelling
    "post_synaptic_density",
    "stress_granule",
    "chromosome",
    "nucleolus",
    "nuclear_pore_complex",
    "cajal_body",
    "rna_granule",
    "cell_junction",
    "transcriptional",
]

def load_model(snargs):
    model = get_object(snargs.lightning_name, "lightning")(snargs)
    model = model.load_from_checkpoint(
        checkpoint_path = snargs.model_path,
        strict=not snargs.relax_checkpoint_matching,
        **{"args": snargs},
    )
    return model

@torch.no_grad()
def predict_condensates(model, sequences, batch_size=1, round=False):
    scores = []
    for i in range(0, len(sequences), batch_size):
        batch = sequences[i:i+batch_size]
        out = model.model({"x": batch})
        s = torch.sigmoid(out['logit']).to("cpu")
        scores.append(s)
    scores = torch.vstack(scores)
    return torch.round(scores, decimals=3) if round else scores

# --- Load ProtGPS once ---
args = Namespace(**pickle.load(open(PROTGPS_ARGS_PATH,'rb')))
args.model_path = PROTGPS_CKPT_PATH
args.pretrained_hub_dir = ESM_PRETRAIN_DIR
model = load_model(args).eval().to(device)
print("[Cell5a] ProtGPS model loaded on", device)

# --- Score the MetaPredict-KEPT (disorder filter) file from Cell 4 ---
def _score_col(lbl: str) -> str:
    return f"{str(lbl).upper().replace(' ','').replace('-','')}_Score"

s1_disorder_kept = pd.read_csv(s1_kept_path)  # from Cell 4
# focus on 115aa (defensive)
s1_115 = s1_disorder_kept[s1_disorder_kept["tile_len"] == STAGE1_TILE_LEN].copy()
started_115 = len(s1_115)

print(f"[Cell5a] 115aa tiles coming into ProtGPS (after disorder filter): {started_115:,}")

seqs = s1_115["tile_seq"].astype(str).tolist()
if len(seqs):
    scores_t = predict_condensates(model, sequences=seqs, batch_size=PROTGPS_BATCH_SIZE, round=False)
    scores = scores_t.cpu().numpy() if hasattr(scores_t, "cpu") else np.asarray(scores_t)
    if scores.shape != (len(s1_115), len(COMPARTMENT_CLASSES)):
        raise RuntimeError(f"ProtGPS shape mismatch: {scores.shape} vs ({len(s1_115)}, {len(COMPARTMENT_CLASSES)})")
    for j, cls in enumerate(COMPARTMENT_CLASSES):
        s1_115[_score_col(cls)] = scores[:, j]

# Add truth label/score
def _truth_col(uid: str):
    t = TRUTH_MAP.get(str(uid))
    return None if not t else _score_col(t)

s1_115["TRUTH_Label"] = s1_115["uniprot_id"].map(TRUTH_MAP).astype("string")
s1_115["TRUTH_Score"] = s1_115.apply(lambda r: r.get(_truth_col(r["uniprot_id"]), float("nan")), axis=1)

# Write a scored checkpoint just for 115
s1_scored_115_path = Path(s1_kept_path).with_name(Path(s1_kept_path).stem + "_PROTGPS_SCORED_ONLY115.csv")
s1_115.to_csv(s1_scored_115_path, index=False)
print(f"[Cell5a] Wrote: {s1_scored_115_path.name}")
try:
    from IPython.display import display
    display(s1_115.head(5))
except Exception:
    pass
#took this many mins:

Using ESM hidden layers 6
Using ESM hidden layers 6
[Cell5a] ProtGPS model loaded on cpu


Using cache found in /users/marcellamirabelli/protgps/esm_models/esm2/facebookresearch_esm_main
Using cache found in /users/marcellamirabelli/protgps/esm_models/esm2/facebookresearch_esm_main


[Cell5a] 115aa tiles coming into ProtGPS (after disorder filter): 45,440
[Cell5a] Wrote: stage1_len115_stride1_metapredict_KEPT_mean0p5_PROTGPS_SCORED_ONLY115.csv


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,disorder_mean,NUCLEAR_SPECKLE_Score,...,POST_SYNAPTIC_DENSITY_Score,STRESS_GRANULE_Score,CHROMOSOME_Score,NUCLEOLUS_Score,NUCLEAR_PORE_COMPLEX_Score,CAJAL_BODY_Score,RNA_GRANULE_Score,CELL_JUNCTION_Score,TRANSCRIPTIONAL_Score,TRUTH_Score
0,Q9C0C2,115,1,0,0,115,MKVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRA...,chromosome,0.903492,2.579210e-11,...,3.117380e-16,2.468393e-06,0.000003,0.164230,1.362807e-07,6.111725e-08,1.494731e-16,1.808685e-07,4.412044e-09,0.000003
1,Q9C0C2,115,1,0,1,116,KVSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRAL...,chromosome,0.896493,1.336807e-10,...,4.790032e-13,4.489589e-07,0.000072,0.481862,4.695206e-06,5.437054e-06,1.150835e-14,1.164430e-07,1.028242e-08,0.000072
2,Q9C0C2,115,1,0,2,117,VSTLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALP...,chromosome,0.897960,2.228123e-10,...,3.502908e-11,1.813325e-06,0.000005,0.563226,4.015617e-07,4.165806e-06,8.033241e-14,2.451353e-06,2.270040e-08,0.000005
3,Q9C0C2,115,1,0,3,118,STLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPA...,chromosome,0.902610,1.510212e-08,...,7.264289e-09,2.173316e-05,0.000013,0.833110,1.361341e-06,1.199329e-04,2.525037e-12,1.342444e-05,8.117610e-08,0.000013
4,Q9C0C2,115,1,0,4,119,TLRESSAMASPLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAK...,chromosome,0.905113,1.263726e-07,...,2.801131e-08,4.275452e-05,0.000038,0.959706,1.007841e-06,7.469367e-04,9.930649e-11,2.156556e-06,8.301448e-08,0.000038


Sanity check 7: 


checking that the truth label and truth score for this protein is in the correct localization

In [14]:
import numpy as np
import pandas as pd

def _score_col(lbl: str) -> str:
    return f"{str(lbl).upper().replace(' ','').replace('-','')}_Score"

# get all proteins present
uids = s1_115["uniprot_id"].unique()

# choose 3 unique proteins (or fewer if less exist)
sampled_uids = np.random.choice(uids, size=min(3, len(uids)), replace=False)

print(f"[Truth wiring check] Sampling 1 tile each from {len(sampled_uids)} protein(s)")

for uid in sampled_uids:
    sub = s1_115[s1_115["uniprot_id"] == uid]
    row = sub.sample(n=1, random_state=None).iloc[0]  # random tile

    truth = row.get("TRUTH_Label", pd.NA)
    if pd.isna(truth) or not str(truth):
        print(f"- {uid} [{row['start']}-{row['end']}] TRUTH_Label missing → skip")
        continue

    col = _score_col(truth)
    truth_score = row.get("TRUTH_Score", np.nan)
    col_score   = row.get(col, np.nan)

    ok = (
        pd.notna(truth_score) and pd.notna(col_score)
        and np.isclose(float(truth_score), float(col_score), atol=1e-9)
    )
    status = "OK" if ok else "MISMATCH"

    print(
        f"- {uid} [{int(row['start'])}-{int(row['end'])}] "
        f"TRUTH_Label={truth} | TRUTH_Score={truth_score:.6f} | "
        f"{col}={col_score:.6f} → {status}"
    )


[Truth wiring check] Sampling 1 tile each from 3 protein(s)
- Q9BUT9 [1-116] TRUTH_Label=stress_granule | TRUTH_Score=0.000010 | STRESS_GRANULE_Score=0.000010 → OK
- Q96SB4 [299-414] TRUTH_Label=chromosome | TRUTH_Score=0.983023 | CHROMOSOME_Score=0.983023 → OK
- P67809 [172-287] TRUTH_Label=stress_granule | TRUTH_Score=0.938502 | STRESS_GRANULE_Score=0.938502 → OK


Cell 5b:

Applying the ProtGPS filter

In [15]:
# ===== Cell 5b: Apply thresholds (disorder + truth) for Stage-1 115aa =====
import pandas as pd 
from pathlib import Path

# Load the scored 115-only checkpoint from Cell 5a: ProtGPS scoring for Stage-1 (115aa) tiles
s1_scored_115 = pd.read_csv(s1_scored_115_path)

# Safety: ensure required columns exist
req = {"disorder_mean", "TRUTH_Score", "tile_len"}
missing = req - set(s1_scored_115.columns)
if missing:
    raise ValueError(f"[Cell5b] Missing required columns: {missing}")

total_115_scored = len(s1_scored_115)

# Apply BOTH thresholds (even though disorder was already filtered earlier, we include it here for clarity)
s1_115_thresh = s1_scored_115[
    (s1_scored_115["disorder_mean"] > DISORDER_THR) &
    (s1_scored_115["TRUTH_Score"]   >= TRUTH_THR)
].copy()

kept_115_thresholds = len(s1_115_thresh)
thrown_115_thresholds = total_115_scored - kept_115_thresholds

print(f"[Cell5b] 115aa scored total (pre-thresholds):        {total_115_scored:,}")
print(f"[Cell5b] 115aa kept after thresholds "
      f"(dis>{DISORDER_THR}, truth≥{TRUTH_THR}): {kept_115_thresholds:,}")
print(f"[Cell5b] 115aa thrown by thresholds:                 {thrown_115_thresholds:,}")

# Write a thresholds-applied checkpoint
s1_115_after_thr_path = Path(s1_scored_115_path).with_name(Path(s1_scored_115_path).stem + "_AFTER_THRESH.csv")
s1_115_thresh.to_csv(s1_115_after_thr_path, index=False)
print(f"[Cell5b] Wrote: {s1_115_after_thr_path.name}")

try:
    from IPython.display import display
    display(s1_115_thresh.head(10))
except Exception:
    pass


[Cell5b] 115aa scored total (pre-thresholds):        45,440
[Cell5b] 115aa kept after thresholds (dis>0.5, truth≥0.65): 8,873
[Cell5b] 115aa thrown by thresholds:                 36,567
[Cell5b] Wrote: stage1_len115_stride1_metapredict_KEPT_mean0p5_PROTGPS_SCORED_ONLY115_AFTER_THRESH.csv


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,disorder_mean,NUCLEAR_SPECKLE_Score,...,POST_SYNAPTIC_DENSITY_Score,STRESS_GRANULE_Score,CHROMOSOME_Score,NUCLEOLUS_Score,NUCLEAR_PORE_COMPLEX_Score,CAJAL_BODY_Score,RNA_GRANULE_Score,CELL_JUNCTION_Score,TRANSCRIPTIONAL_Score,TRUTH_Score
14,Q9C0C2,115,1,0,14,129,PLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLL...,chromosome,0.911841,1.704135e-04,...,2.815164e-08,0.000004,0.683532,0.028905,0.000011,0.000081,4.419953e-09,6.687606e-10,1.589457e-06,0.683532
80,Q9C0C2,115,1,0,80,195,MNMLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEE...,chromosome,0.841084,1.364906e-07,...,2.268099e-09,0.000060,0.987630,0.000452,0.001225,0.000035,1.646159e-13,1.492708e-04,1.749660e-06,0.987630
81,Q9C0C2,115,1,0,81,196,NMLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEP...,chromosome,0.831283,2.061721e-08,...,4.230499e-11,0.000946,0.992560,0.000479,0.001288,0.000002,9.472461e-14,4.908066e-06,9.395966e-07,0.992561
82,Q9C0C2,115,1,0,82,197,MLAGPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPP...,chromosome,0.826323,2.424349e-07,...,7.190127e-09,0.018205,0.962125,0.002346,0.070082,0.000182,1.907509e-10,7.316496e-06,9.546202e-06,0.962125
83,Q9C0C2,115,1,0,83,198,LAGPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPP...,chromosome,0.824310,4.424250e-06,...,4.283641e-07,0.004724,0.723008,0.220417,0.150414,0.009285,8.353020e-08,9.127798e-04,2.760455e-05,0.723008
84,Q9C0C2,115,1,0,84,199,AGPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPPL...,chromosome,0.831096,5.429240e-06,...,1.089299e-09,0.000285,0.943809,0.236212,0.039121,0.001618,2.852767e-09,2.219211e-03,7.289933e-06,0.943809
85,Q9C0C2,115,1,0,85,200,GPQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPPLT...,chromosome,0.836766,8.152744e-06,...,3.801001e-11,0.000129,0.985472,0.211517,0.014566,0.000318,5.929288e-10,1.742805e-03,4.317948e-06,0.985472
86,Q9C0C2,115,1,0,86,201,PQPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPPLTP...,chromosome,0.832440,4.518838e-05,...,5.751539e-12,0.000393,0.989893,0.093205,0.003959,0.000323,1.285615e-09,1.355357e-04,1.413693e-05,0.989893
87,Q9C0C2,115,1,0,87,202,QPYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPPLTPP...,chromosome,0.788817,3.610421e-05,...,3.941558e-12,0.000355,0.996371,0.035137,0.003095,0.000087,2.988429e-11,9.021413e-05,2.813976e-06,0.996371
88,Q9C0C2,115,1,0,88,203,PYGGSKRPLPFAPRPAVEASTGGEATQETGKEEAGKEEPPPLTPPA...,chromosome,0.798557,1.191344e-04,...,5.587839e-11,0.000097,0.997465,0.364437,0.001035,0.000102,2.497190e-09,5.689110e-05,2.050297e-06,0.997465


Cell 5c: 

Filtering for redundancy of sequence within the tiling library

In [16]:
# ===== Cell 5c: N→C contiguous-run dedupe for 115aa tiles =====
import pandas as pd
import numpy as np
from pathlib import Path

# Input from Cell 5b
df_in = s1_115_thresh.copy()
before = len(df_in)

# --- knob: contiguous similarity as fraction of tile length ---
SIM_THR_FRAC = 0.25   # e.g., 0.25 = 25% of L (115) → 29 aa
L = STAGE1_TILE_LEN
RUN_THR = int(np.ceil(SIM_THR_FRAC * L))  # minimum contiguous identical run to call "redundant"

def lcs_contig_len(a: str, b: str) -> int:
    """Length of the longest contiguous identical substring between a and b (O(L^2), L≈115)."""
    if not a or not b or len(a) != len(b):  # tiles are equal length by construction, but be safe
        return 0
    la = len(a); lb = len(b)
    prev = [0] * (lb + 1)
    best = 0
    for i in range(1, la + 1):
        ai = a[i - 1]
        curr = [0] * (lb + 1)
        for j in range(1, lb + 1):
            if ai == b[j - 1]:
                v = prev[j - 1] + 1
                curr[j] = v
                if v > best:
                    best = v
        prev = curr
    return best

def dedupe_contiguous_n2c(df: pd.DataFrame, run_thr: int) -> pd.DataFrame:
    req = {"uniprot_id","tile_len","start","tile_seq"}
    miss = req - set(df.columns)
    if miss:
        raise ValueError(f"[dedupe] Missing columns: {miss}")
    out_frames = []
    for uid, grp in df.groupby("uniprot_id", sort=False):
        g = grp[grp["tile_len"] == L].sort_values("start", kind="stable").reset_index(drop=True)
        if g.empty:
            continue
        kept_rows = []
        kept_seqs = []
        for _, row in g.iterrows():
            s = row["tile_seq"]
            redundant = False
            for ks in kept_seqs:
                if lcs_contig_len(s, ks) >= run_thr:
                    redundant = True
                    break
            if redundant:
                continue
            kept_rows.append(row)
            kept_seqs.append(s)
        out_frames.append(pd.DataFrame(kept_rows))
    return pd.concat(out_frames, ignore_index=True) if out_frames else df.head(0).copy()

s1_115_deduped = dedupe_contiguous_n2c(df_in, RUN_THR)
after = len(s1_115_deduped)
print(f"[Cell5c] L={L}, RUN_THR={RUN_THR} aa (≈{SIM_THR_FRAC:.0%} of L)")
print(f"[Cell5c] before: {before:,}  after: {after:,}  dropped: {before-after:,}")

# Save checkpoint
s1_115_deduped_path = Path(s1_115_after_thr_path).with_name(
    Path(s1_115_after_thr_path).stem + f"_AFTER_CONRUN{RUN_THR}.csv"
)
s1_115_deduped.to_csv(s1_115_deduped_path, index=False)
print(f"[Cell5c] Wrote: {s1_115_deduped_path.name}")


[Cell5c] L=115, RUN_THR=29 aa (≈25% of L)
[Cell5c] before: 8,873  after: 279  dropped: 8,594
[Cell5c] Wrote: stage1_len115_stride1_metapredict_KEPT_mean0p5_PROTGPS_SCORED_ONLY115_AFTER_THRESH_AFTER_CONRUN29.csv


In [17]:
import pandas as pd

print(f"[115 Library] total tiles: {len(s1_115_deduped):,}")
display(s1_115_deduped.head(20))   # first 20 rows


[115 Library] total tiles: 279


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,disorder_mean,NUCLEAR_SPECKLE_Score,...,POST_SYNAPTIC_DENSITY_Score,STRESS_GRANULE_Score,CHROMOSOME_Score,NUCLEOLUS_Score,NUCLEAR_PORE_COMPLEX_Score,CAJAL_BODY_Score,RNA_GRANULE_Score,CELL_JUNCTION_Score,TRANSCRIPTIONAL_Score,TRUTH_Score
0,Q9C0C2,115,1,0,14,129,PLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLL...,chromosome,0.911841,1.704135e-04,...,2.815164e-08,4.453621e-06,0.683532,2.890525e-02,1.149208e-05,8.059703e-05,4.419953e-09,6.687606e-10,1.589457e-06,0.683532
1,Q9C0C2,115,1,0,193,308,SSRYGPRTYGTTTAPRDEDGSTLFRGWSQEGPVKSPAECREEHSKT...,chromosome,0.935478,3.352584e-05,...,2.003914e-08,3.593793e-02,0.673018,9.033319e-03,1.823470e-01,1.754689e-02,8.341208e-05,1.559996e-05,6.819663e-05,0.673018
2,Q9C0C2,115,1,0,314,429,SQLLEAQTPEASQASPCPAVTPSAPSAALPDEGSRHTPSPGLPAEG...,chromosome,0.912317,4.311244e-05,...,1.386831e-10,2.141602e-09,0.888445,1.223208e-04,8.429254e-04,2.245317e-07,6.856866e-16,7.947126e-01,5.058073e-07,0.888445
3,Q9C0C2,115,1,0,401,516,RTFPSGGEEEAKGDAHLRPTSLVQRRFSEGVLQSPSQDQEKLGGSL...,chromosome,0.945088,1.520144e-10,...,5.630878e-07,5.356580e-06,0.998066,1.111183e-02,9.070813e-02,2.583631e-08,2.084085e-14,3.288412e-07,5.084631e-08,0.998066
4,Q9C0C2,115,1,0,488,603,VWRLDSPPPSPITEASEAAEAAEAGNLAVSSREEGVSQQGQGAGSA...,chromosome,0.946219,9.572823e-06,...,1.834590e-08,1.348398e-03,0.999807,1.365985e-04,9.509306e-04,6.124374e-08,2.075454e-13,2.814670e-08,2.713091e-08,0.999807
5,Q9C0C2,115,1,0,694,809,SGGGARRGAGAELKDTQSPSTCSEGLLGWSQKDLQSEFGITGDPQP...,chromosome,0.930002,1.036747e-06,...,6.713979e-05,3.434311e-01,0.789456,1.647750e-04,2.218575e-03,1.681724e-07,6.519803e-07,4.278671e-04,2.213772e-05,0.789456
6,Q9C0C2,115,1,0,961,1076,SSGGSSRTLDAQDRSFGTRPLSSGFSPEEAQQQDEEFEKKIPSVED...,chromosome,0.942183,2.003533e-05,...,1.997279e-08,6.653707e-03,0.832553,8.278351e-06,6.394283e-03,5.486609e-08,5.150047e-11,3.831590e-01,8.327408e-08,0.832553
7,Q9C0C2,115,1,0,1168,1283,NLEVSSCVGSGGSSEARESAVGQMGWSGGLSLRDMNLTGCLESGGS...,chromosome,0.938375,1.792185e-10,...,2.499806e-13,2.257120e-02,0.973179,1.589207e-01,7.189431e-04,8.027473e-09,2.122660e-15,1.400975e-07,9.192796e-11,0.973179
8,Q9C0C2,115,1,0,1292,1407,GAVCSPGESKELGVGQMDWGNNLGLRDLEVTCDPDSGGSQGLRGCG...,chromosome,0.945552,1.833349e-14,...,7.567247e-17,1.252387e-02,0.730168,2.618078e-06,1.382923e-03,4.423039e-13,1.697367e-18,1.236177e-05,4.222271e-10,0.730168
9,Q9C0C2,115,1,0,1380,1495,NGSLSPGLEARDPLEARELGVGETSGPETQGEDYSSSSLEPHPADP...,chromosome,0.913027,1.479464e-02,...,2.094478e-06,1.281299e-07,0.701970,3.725457e-03,1.085313e-06,5.736471e-06,8.519258e-13,4.236026e-03,5.880929e-08,0.701970


Summary of Stage 1: generating the 115aa library and setting up for stage 2


each parent tile yields: 115-66+1= 50 tiles per stage 1 tile

In [18]:
# ===== Sanity check: final 115aa library and expected Stage-2 tiling =====
import pandas as pd

# (assumes s1_115_deduped already exists from Cell 5c)
df_final_115 = s1_115_deduped.copy()

# --- 1. Output the final 115aa library
print(f"[Sanity] Final 115aa library shape: {df_final_115.shape}")
display(df_final_115.head(10))   # first 10 rows

# --- 2. How many 115aa tiles did we keep?
num_115 = len(df_final_115)
print(f"[Sanity] Total kept 115aa tiles: {num_115:,}")

# --- 3. Expected 66aa tiles from Stage-2 (stride=1) ---
def expected_tiles(n, L, stride=1):
    """How many tiles of length L can be made from seq length n with stride."""
    if n < L: return 0
    return ((n - L) // stride) + 1

L2 = 66
stride2 = 1
expected_total_66 = 0

for seq in df_final_115["tile_seq"]:
    expected_total_66 += expected_tiles(len(seq), L2, stride2)

print(f"[Sanity] Expected 66aa tiles in Stage-2 (stride={stride2}): {expected_total_66:,}")

# Optional: average per parent tile
print(f"[Sanity] On average, {expected_total_66/num_115:.1f} × 66aa tiles per 115aa parent tile")
# --- 4. Number of unique proteins represented in final 115aa library ---
if "uniprot_id" in df_final_115.columns:
    num_unique_proteins = df_final_115["uniprot_id"].nunique()
    print(f"[Sanity] Unique proteins in final 115aa library: {num_unique_proteins:,}")
else:
    print("[Warning] No 'uniprot_id' column found in df_final_115")


[Sanity] Final 115aa library shape: (279, 22)


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,disorder_mean,NUCLEAR_SPECKLE_Score,...,POST_SYNAPTIC_DENSITY_Score,STRESS_GRANULE_Score,CHROMOSOME_Score,NUCLEOLUS_Score,NUCLEAR_PORE_COMPLEX_Score,CAJAL_BODY_Score,RNA_GRANULE_Score,CELL_JUNCTION_Score,TRANSCRIPTIONAL_Score,TRUTH_Score
0,Q9C0C2,115,1,0,14,129,PLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLL...,chromosome,0.911841,1.704135e-04,...,2.815164e-08,4.453621e-06,0.683532,0.028905,0.000011,8.059703e-05,4.419953e-09,6.687606e-10,1.589457e-06,0.683532
1,Q9C0C2,115,1,0,193,308,SSRYGPRTYGTTTAPRDEDGSTLFRGWSQEGPVKSPAECREEHSKT...,chromosome,0.935478,3.352584e-05,...,2.003914e-08,3.593793e-02,0.673018,0.009033,0.182347,1.754689e-02,8.341208e-05,1.559996e-05,6.819663e-05,0.673018
2,Q9C0C2,115,1,0,314,429,SQLLEAQTPEASQASPCPAVTPSAPSAALPDEGSRHTPSPGLPAEG...,chromosome,0.912317,4.311244e-05,...,1.386831e-10,2.141602e-09,0.888445,0.000122,0.000843,2.245317e-07,6.856866e-16,7.947126e-01,5.058073e-07,0.888445
3,Q9C0C2,115,1,0,401,516,RTFPSGGEEEAKGDAHLRPTSLVQRRFSEGVLQSPSQDQEKLGGSL...,chromosome,0.945088,1.520144e-10,...,5.630878e-07,5.356580e-06,0.998066,0.011112,0.090708,2.583631e-08,2.084085e-14,3.288412e-07,5.084631e-08,0.998066
4,Q9C0C2,115,1,0,488,603,VWRLDSPPPSPITEASEAAEAAEAGNLAVSSREEGVSQQGQGAGSA...,chromosome,0.946219,9.572823e-06,...,1.834590e-08,1.348398e-03,0.999807,0.000137,0.000951,6.124374e-08,2.075454e-13,2.814670e-08,2.713091e-08,0.999807
5,Q9C0C2,115,1,0,694,809,SGGGARRGAGAELKDTQSPSTCSEGLLGWSQKDLQSEFGITGDPQP...,chromosome,0.930002,1.036747e-06,...,6.713979e-05,3.434311e-01,0.789456,0.000165,0.002219,1.681724e-07,6.519803e-07,4.278671e-04,2.213772e-05,0.789456
6,Q9C0C2,115,1,0,961,1076,SSGGSSRTLDAQDRSFGTRPLSSGFSPEEAQQQDEEFEKKIPSVED...,chromosome,0.942183,2.003533e-05,...,1.997279e-08,6.653707e-03,0.832553,0.000008,0.006394,5.486609e-08,5.150047e-11,3.831590e-01,8.327408e-08,0.832553
7,Q9C0C2,115,1,0,1168,1283,NLEVSSCVGSGGSSEARESAVGQMGWSGGLSLRDMNLTGCLESGGS...,chromosome,0.938375,1.792185e-10,...,2.499806e-13,2.257120e-02,0.973179,0.158921,0.000719,8.027473e-09,2.122660e-15,1.400975e-07,9.192796e-11,0.973179
8,Q9C0C2,115,1,0,1292,1407,GAVCSPGESKELGVGQMDWGNNLGLRDLEVTCDPDSGGSQGLRGCG...,chromosome,0.945552,1.833349e-14,...,7.567247e-17,1.252387e-02,0.730168,0.000003,0.001383,4.423039e-13,1.697367e-18,1.236177e-05,4.222271e-10,0.730168
9,Q9C0C2,115,1,0,1380,1495,NGSLSPGLEARDPLEARELGVGETSGPETQGEDYSSSSLEPHPADP...,chromosome,0.913027,1.479464e-02,...,2.094478e-06,1.281299e-07,0.701970,0.003725,0.000001,5.736471e-06,8.519258e-13,4.236026e-03,5.880929e-08,0.701970


[Sanity] Total kept 115aa tiles: 279
[Sanity] Expected 66aa tiles in Stage-2 (stride=1): 13,950
[Sanity] On average, 50.0 × 66aa tiles per 115aa parent tile
[Sanity] Unique proteins in final 115aa library: 78


Stage 2: generating 66aa tiles from the 115aa library

Sanity check for deduplication:
print each proteins tiles and making sure that each protein has different enough tiles 

In [19]:
# ===== Cell 6a: Re-tile Stage-1 (deduped 115aa) into 66aa tiles =====
import pandas as pd
from pathlib import Path

# read the deduped 115-aa tiles from Cell 5c
s1_115_deduped = pd.read_csv(s1_115_deduped_path)

# define tile_sequence if not already in the notebook
if "tile_sequence" not in globals():
    def tile_sequence(seq, tile_len, stride=1, phase=0):
        n = len(seq)
        if tile_len > n:
            return []
        return [(i, seq[i:i+tile_len]) for i in range(phase, n - tile_len + 1, stride)]

def retile_kept_stage1(df_kept: pd.DataFrame, sub_lengths, sub_stride: int) -> pd.DataFrame:
    out = []
    for _, r in df_kept.iterrows():
        uid   = str(r["uniprot_id"])
        s1    = int(r["start"])               # start of the 115-aa parent in protein coords
        tseq1 = str(r["tile_seq"])            # sequence of the 115-aa parent
        parent_len = int(r["tile_len"])       # should be 115

        for L2 in sub_lengths:
            if L2 > parent_len:
                continue
            for sub_start, subseq in tile_sequence(tseq1, tile_len=L2, stride=sub_stride, phase=0):
                start2 = s1 + sub_start       # convert back to protein coordinates
                out.append({
                    "uniprot_id":   uid,
                    "tile_len":     L2,
                    "stride":       sub_stride,
                    "phase":        0,
                    "start":        start2,
                    "end":          start2 + L2,
                    "tile_seq":     subseq,
                    "TRUTH_Label":  TRUTH_MAP.get(uid),
                    "parent_len":   parent_len,
                    "parent_start": s1,
                })
    return pd.DataFrame(out)

S2_DESC = f"stage2_from_stage1_len{STAGE1_TILE_LEN}_sub{'-'.join(map(str,STAGE2_TILE_LENGTHS))}_stride{STAGE2_STRIDE}"
S2_BASE = RUN_DIR / S2_DESC

s2_tiles = retile_kept_stage1(s1_115_deduped, STAGE2_TILE_LENGTHS, STAGE2_STRIDE)
s2_tiles_path = S2_BASE.with_suffix(".tiles.csv")
s2_tiles.to_csv(s2_tiles_path, index=False)

print(f"[Cell6a] Stage-2 tiles written: {len(s2_tiles):,}")
print(f"[Cell6a] File: {s2_tiles_path.name}")
try:
    from IPython.display import display
    display(s2_tiles.head(5))
except Exception:
    pass


[Cell6a] Stage-2 tiles written: 13,950
[Cell6a] File: stage2_from_stage1_len115_sub66_stride1.tiles.csv


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,parent_len,parent_start
0,Q9C0C2,66,1,0,14,80,PLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLL...,chromosome,115,14
1,Q9C0C2,66,1,0,15,81,LPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLV...,chromosome,115,14
2,Q9C0C2,66,1,0,16,82,PREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVP...,chromosome,115,14
3,Q9C0C2,66,1,0,17,83,REMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPV...,chromosome,115,14
4,Q9C0C2,66,1,0,18,84,EMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVG...,chromosome,115,14


In [20]:
# ===== Cell 6b: MetaPredict on Stage-2 tiles (RESULTS + KEPT) =====
from pathlib import Path

# reuse the run_metapredict defined in Cell 4
s2_results_path, s2_kept_path = run_metapredict(s2_tiles_path, S2_BASE, DISORDER_THR)

# quick counts for clarity
import pandas as pd
s2_results = pd.read_csv(s2_results_path)
s2_kept    = pd.read_csv(s2_kept_path)

print(f"[Cell6b] Stage-2 tiles total (scored): {len(s2_results):,}")
print(f"[Cell6b] Stage-2 kept by disorder (>{DISORDER_THR}): {len(s2_kept):,}")
print(f"[Cell6b] Stage-2 thrown by disorder: {len(s2_results) - len(s2_kept):,}")
print(f"[Cell6b] Files: {s2_results_path.name}, {s2_kept_path.name}")
try:
    from IPython.display import display
    display(s2_kept.head(5))
except Exception:
    pass


[Cell4][MetaPredict CMD] /opt/anaconda3/bin/conda run -n metapredict_env python -c 
import pandas as pd, numpy as np, sys
try:
    import metapredict as mp
except Exception as e:
    raise SystemExit("ERROR: metapredict not available in this environment") from e
inp, out_res, out_kept, thr = sys.argv[1], sys.argv[2], sys.argv[3], float(sys.argv[4])
df = pd.read_csv(inp)
need = {"uniprot_id","tile_len","stride","phase","start","end","tile_seq"}
missing = need - set(df.columns)
if missing:
    raise SystemExit(f"Input CSV missing columns: {missing}")
def avg_disorder(seq: str) -> float:
    if not isinstance(seq, str) or not seq: return 0.0
    scores = np.asarray(mp.predict_disorder(seq), dtype=float)
    return float(scores.mean()) if scores.size else 0.0
try:
    from tqdm.auto import tqdm
    tqdm.pandas(desc="MetaPredict (mean disorder)")
    df["disorder_mean"] = df["tile_seq"].progress_apply(avg_disorder)
except Exception:
    df["disorder_mean"] = [avg_disorder(s) for s in df["ti

,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,parent_len,parent_start,disorder_mean
0,Q9C0C2,66,1,0,14,80,PLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLL...,chromosome,115,14,0.887953
1,Q9C0C2,66,1,0,15,81,LPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLV...,chromosome,115,14,0.888100
2,Q9C0C2,66,1,0,16,82,PREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVP...,chromosome,115,14,0.893150
3,Q9C0C2,66,1,0,17,83,REMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPV...,chromosome,115,14,0.898017
4,Q9C0C2,66,1,0,18,84,EMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVG...,chromosome,115,14,0.878080


In [21]:
# ===== Cell 6c: ProtGPS on Stage-2 (66aa) KEPT + thresholds =====
import pandas as pd, numpy as np
from pathlib import Path

# Reuse: predict_condensates(model,...), COMPARTMENT_CLASSES, TRUTH_MAP, PROTGPS_BATCH_SIZE
def _score_col(lbl: str) -> str:
    return f"{str(lbl).upper().replace(' ','').replace('-','')}_Score"

# 1) Load disorder-kept 66-aa tiles and score with ProtGPS
s2_kept_df = pd.read_csv(s2_kept_path)
seqs = s2_kept_df["tile_seq"].astype(str).tolist()

if len(seqs):
    scores_t = predict_condensates(model, sequences=seqs, batch_size=PROTGPS_BATCH_SIZE, round=False)
    scores = scores_t.cpu().numpy() if hasattr(scores_t, "cpu") else np.asarray(scores_t)
    if scores.shape != (len(s2_kept_df), len(COMPARTMENT_CLASSES)):
        raise RuntimeError(f"[Cell6c] ProtGPS shape mismatch: {scores.shape} vs ({len(s2_kept_df)}, {len(COMPARTMENT_CLASSES)})")
    for j, cls in enumerate(COMPARTMENT_CLASSES):
        s2_kept_df[_score_col(cls)] = scores[:, j]

# 2) Add TRUTH_Label/TRUTH_Score columns
def _truth_col(uid: str):
    t = TRUTH_MAP.get(str(uid))
    return None if not t else _score_col(t)

s2_kept_df["TRUTH_Label"] = s2_kept_df["uniprot_id"].map(TRUTH_MAP).astype("string")
s2_kept_df["TRUTH_Score"] = s2_kept_df.apply(lambda r: r.get(_truth_col(r["uniprot_id"]), float("nan")), axis=1)

# 3) Save a scored checkpoint (pre-thresholds)
s2_scored_path = s2_kept_path.with_name(s2_kept_path.stem + "_PROTGPS_SCORED.csv")
s2_kept_df.to_csv(s2_scored_path, index=False)
print(f"[Cell6c] Wrote: {s2_scored_path.name}")

# 4) Apply BOTH thresholds and report counts
total_scored_66 = len(s2_kept_df)
s2_final_keep = s2_kept_df[
    (s2_kept_df["disorder_mean"] > DISORDER_THR) &
    (s2_kept_df["TRUTH_Score"]   >= TRUTH_THR)
].copy()

kept_66 = len(s2_final_keep)
thrown_66 = total_scored_66 - kept_66

print(f"[Cell6c] Stage-2 66aa scored total (pre-thresholds): {total_scored_66:,}")
print(f"[Cell6c] 66aa kept after thresholds "
      f"(dis>{DISORDER_THR}, truth≥{TRUTH_THR}): {kept_66:,}")
print(f"[Cell6c] 66aa thrown by thresholds: {thrown_66:,}")

try:
    from IPython.display import display
    display(s2_final_keep.head(5))
except Exception:
    pass


[Cell6c] Wrote: stage2_from_stage1_len115_sub66_stride1_metapredict_KEPT_mean0p5_PROTGPS_SCORED.csv
[Cell6c] Stage-2 66aa scored total (pre-thresholds): 12,924
[Cell6c] 66aa kept after thresholds (dis>0.5, truth≥0.65): 4,428
[Cell6c] 66aa thrown by thresholds: 8,496


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,parent_len,parent_start,...,POST_SYNAPTIC_DENSITY_Score,STRESS_GRANULE_Score,CHROMOSOME_Score,NUCLEOLUS_Score,NUCLEAR_PORE_COMPLEX_Score,CAJAL_BODY_Score,RNA_GRANULE_Score,CELL_JUNCTION_Score,TRANSCRIPTIONAL_Score,TRUTH_Score
0,Q9C0C2,66,1,0,14,80,PLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLL...,chromosome,115,14,...,5.214741e-11,8.469487e-09,0.981388,0.900916,1.644538e-07,0.000870,9.772376e-12,4.084947e-11,2.408852e-08,0.981388
1,Q9C0C2,66,1,0,15,81,LPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLV...,chromosome,115,14,...,6.812938e-11,6.636249e-09,0.942057,0.859800,3.367650e-07,0.001652,1.068944e-11,3.697170e-11,1.525344e-08,0.942057
2,Q9C0C2,66,1,0,16,82,PREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVP...,chromosome,115,14,...,4.555944e-13,3.523934e-06,0.821565,0.861061,1.022081e-07,0.002563,6.586702e-09,6.065553e-11,4.966253e-07,0.821565
3,Q9C0C2,66,1,0,17,83,REMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPV...,chromosome,115,14,...,7.286081e-15,4.446612e-07,0.759487,0.978672,7.615876e-08,0.000238,5.804427e-12,1.572810e-11,9.848004e-09,0.759487
4,Q9C0C2,66,1,0,18,84,EMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLLVPVG...,chromosome,115,14,...,2.174167e-15,1.023091e-06,0.653756,0.970751,1.835780e-07,0.000524,1.198117e-13,6.129752e-11,1.000826e-08,0.653756


Cell 6d: 

deduplicating from N-C terminus for stage 2 66aa tiles

In [22]:
# ===== Cell 6d (revised): N→C contiguous-run dedupe for 66-aa tiles =====
import pandas as pd
import numpy as np
from pathlib import Path

# Input: the Stage-2 kept/scored set before dedupe
df_in = s2_final_keep.copy()
before = len(df_in)

# --- knob: contiguous similarity as fraction of tile length ---
SIM_THR_FRAC = 0.25       # e.g., 0.25 of 66 → 17 aa
L = 66                    # Stage-2 tile length
RUN_THR = int(np.ceil(SIM_THR_FRAC * L))  # minimum contiguous identical run to call "redundant"

def lcs_contig_len(a: str, b: str) -> int:
    """Length of the longest contiguous identical substring between a and b (O(L^2), L≈66)."""
    if not a or not b or len(a) != len(b):  # equal length tiles expected; be defensive
        return 0
    la, lb = len(a), len(b)
    prev = [0] * (lb + 1)
    best = 0
    for i in range(1, la + 1):
        ai = a[i - 1]
        curr = [0] * (lb + 1)
        for j in range(1, lb + 1):
            if ai == b[j - 1]:
                v = prev[j - 1] + 1
                curr[j] = v
                if v > best:
                    best = v
        prev = curr
    return best

def dedupe_contiguous_n2c_66(df: pd.DataFrame, run_thr: int) -> pd.DataFrame:
    req = {"uniprot_id","tile_len","start","tile_seq"}
    miss = req - set(df.columns)
    if miss:
        raise ValueError(f"[Cell6d] Missing columns: {miss}")

    out_frames = []
    for uid, grp in df.groupby("uniprot_id", sort=False):
        g = grp[grp["tile_len"] == L].sort_values("start", kind="stable").reset_index(drop=True)
        if g.empty:
            continue
        kept_rows, kept_seqs = [], []
        for _, row in g.iterrows():
            s = row["tile_seq"]
            redundant = any(lcs_contig_len(s, ks) >= run_thr for ks in kept_seqs)
            if redundant:
                continue
            kept_rows.append(row)
            kept_seqs.append(s)
        out_frames.append(pd.DataFrame(kept_rows))
    return pd.concat(out_frames, ignore_index=True) if out_frames else df.head(0).copy()

# Run dedupe
s2_66_deduped = dedupe_contiguous_n2c_66(df_in, RUN_THR)
after = len(s2_66_deduped)
print(f"[Cell6d] L={L}, RUN_THR={RUN_THR} aa (≈{SIM_THR_FRAC:.0%} of L)")
print(f"[Cell6d] 66-aa before dedupe: {before:,}")
print(f"[Cell6d] 66-aa after  dedupe : {after:,}")
print(f"[Cell6d] 66-aa dropped       : {before-after:,}")

# Write checkpoint and promote
s2_66_deduped_path = Path(s2_scored_path).with_name(
    Path(s2_scored_path).stem + f"_AFTER_CONRUN{RUN_THR}.csv"
)
s2_66_deduped.to_csv(s2_66_deduped_path, index=False)
print(f"[Cell6d] Wrote: {s2_66_deduped_path.name}")

# expose for downstream steps
s2_final_keep = s2_66_deduped

try:
    from IPython.display import display
    display(s2_final_keep.head(5))
except Exception:
    pass


[Cell6d] L=66, RUN_THR=17 aa (≈25% of L)
[Cell6d] 66-aa before dedupe: 4,428
[Cell6d] 66-aa after  dedupe : 222
[Cell6d] 66-aa dropped       : 4,206
[Cell6d] Wrote: stage2_from_stage1_len115_sub66_stride1_metapredict_KEPT_mean0p5_PROTGPS_SCORED_AFTER_CONRUN17.csv


,uniprot_id,tile_len,stride,phase,start,end,tile_seq,TRUTH_Label,parent_len,parent_start,...,POST_SYNAPTIC_DENSITY_Score,STRESS_GRANULE_Score,CHROMOSOME_Score,NUCLEOLUS_Score,NUCLEAR_PORE_COMPLEX_Score,CAJAL_BODY_Score,RNA_GRANULE_Score,CELL_JUNCTION_Score,TRANSCRIPTIONAL_Score,TRUTH_Score
0,Q9C0C2,66,1,0,14,80,PLPREMEEELVPTGSEPGDTRAKPPVKPKPRALPAKPALPAKPSLL...,chromosome,115,14,...,5.214741e-11,8.469487e-09,0.981388,0.900916,1.644538e-07,8.700283e-04,9.772376e-12,4.084947e-11,2.408852e-08,0.981388
1,Q9C0C2,66,1,0,195,261,RYGPRTYGTTTAPRDEDGSTLFRGWSQEGPVKSPAECREEHSKTPE...,chromosome,115,193,...,1.105199e-11,2.707705e-01,0.784753,0.034491,8.364632e-04,1.205501e-03,9.637424e-05,2.845251e-07,4.768772e-06,0.784753
2,Q9C0C2,66,1,0,320,386,QTPEASQASPCPAVTPSAPSAALPDEGSRHTPSPGLPAEGAPEAPR...,chromosome,115,314,...,1.144401e-13,2.883054e-07,0.831399,0.006922,1.415830e-04,5.326645e-07,1.363973e-12,1.987247e-02,7.995692e-08,0.831399
3,Q9C0C2,66,1,0,401,467,RTFPSGGEEEAKGDAHLRPTSLVQRRFSEGVLQSPSQDQEKLGGSL...,chromosome,115,401,...,1.531214e-09,8.355175e-11,0.999817,0.087963,1.706038e-02,8.816760e-08,4.793414e-20,1.756551e-12,1.934198e-11,0.999817
4,Q9C0C2,66,1,0,488,554,VWRLDSPPPSPITEASEAAEAAEAGNLAVSSREEGVSQQGQGAGSA...,chromosome,115,488,...,3.399276e-06,3.665702e-06,0.996448,0.000075,2.005066e-03,2.046808e-08,7.053188e-15,4.924841e-04,2.328082e-08,0.996448


In [23]:
# ===== FINAL SUMMARY: libraries, params, counts =====
import pandas as pd
import numpy as np

def _maybe_load(df_name: str, path_name: str):
    """Return a DataFrame from globals()[df_name] if present, else read from globals()[path_name] if present; else empty df."""
    g = globals()
    if df_name in g and isinstance(g[df_name], pd.DataFrame):
        return g[df_name].copy()
    if path_name in g:
        try:
            return pd.read_csv(g[path_name])
        except Exception:
            pass
    return pd.DataFrame()

def _stage_summary(df, *, tile_len, stride, disorder_thr, truth_thr, sim_thr_frac, label):
    out = {}
    out["label"]          = label
    out["tile_len"]       = tile_len
    out["stride"]         = stride
    out["disorder_thr"]   = disorder_thr
    out["truth_thr"]      = truth_thr
    out["sim_thr_frac"]   = sim_thr_frac
    out["run_thr_aa"]     = int(np.ceil(sim_thr_frac * tile_len)) if sim_thr_frac is not None else None
    out["tiles_final"]    = len(df)
    out["proteins_final"] = df["uniprot_id"].nunique() if not df.empty and "uniprot_id" in df.columns else 0
    return out

def _print_stage(out):
    print(f"\n[{out['label']}]")
    print(f"  Params:")
    print(f"    • tile_len={out['tile_len']}, stride={out['stride']}")
    print(f"    • disorder_thr={out['disorder_thr']}, truth_thr={out['truth_thr']}")
    if out["sim_thr_frac"] is not None:
        print(f"    • dedupe: contiguous-run ≥ {out['run_thr_aa']} aa (~{int(out['sim_thr_frac']*100)}% of tile)")
    print(f"  Final library:")
    print(f"    • proteins kept: {out['proteins_final']}")
    print(f"    • tiles kept   : {out['tiles_final']:,}")

# -------- Load final libraries (prefer in-memory; else from disk) --------
# Stage 1 final (115aa) after thresholds + dedupe
df_115 = _maybe_load("s1_115_deduped", "s1_115_deduped_path")

# Stage 2 final (66aa) after thresholds + dedupe
df_66  = _maybe_load("s2_final_keep", "s2_66_deduped_path")

# -------- Parameters (from your config) --------
# fallbacks in case some constants aren't in scope
STAGE1_TILE_LEN = globals().get("STAGE1_TILE_LEN", 115)
STAGE1_STRIDE   = globals().get("STAGE1_STRIDE", 1)
STAGE2_STRIDE   = globals().get("STAGE2_STRIDE", 1)
DISORDER_THR    = globals().get("DISORDER_THR", 0.50)
TRUTH_THR       = globals().get("TRUTH_THR", 0.65)

# dedupe fractions you used in the notebook (stage1: 0.25; stage2: 0.25)
S1_SIM_FRAC = 0.25
S2_SIM_FRAC = 0.25

# -------- Build & print the summaries --------
s1_out = _stage_summary(
    df_115,
    tile_len=STAGE1_TILE_LEN,
    stride=STAGE1_STRIDE,
    disorder_thr=DISORDER_THR,
    truth_thr=TRUTH_THR,
    sim_thr_frac=S1_SIM_FRAC,
    label="Stage-1 (115aa) library"
)
_print_stage(s1_out)

# Also estimate how many raw 66aa tiles Stage-2 *would* start with from Stage-1 parents
if len(df_115):
    expected_66_from_115 = len(df_115) * (STAGE1_TILE_LEN - 66 + 1)  # 115-66+1 = 50 per parent tile
    print(f"  Expected Stage-2 66aa tiles before Stage-2 filtering: {expected_66_from_115:,}")

s2_out = _stage_summary(
    df_66,
    tile_len=66,
    stride=STAGE2_STRIDE,
    disorder_thr=DISORDER_THR,
    truth_thr=TRUTH_THR,
    sim_thr_frac=S2_SIM_FRAC,
    label="Stage-2 (66aa) library"
)
_print_stage(s2_out)

# -------- Optional: per-protein counts (handy to sanity check balance) --------
if not df_115.empty:
    print("\n[Per-protein counts] Stage-1 (115aa) tiles kept – top 10")
    print(df_115.groupby("uniprot_id").size().sort_values(ascending=False).head(20).to_string())

if not df_66.empty:
    print("\n[Per-protein counts] Stage-2 (66aa) tiles kept – top 10")
    print(df_66.groupby("uniprot_id").size().sort_values(ascending=False).head(20).to_string())



[Stage-1 (115aa) library]
  Params:
    • tile_len=115, stride=1
    • disorder_thr=0.5, truth_thr=0.65
    • dedupe: contiguous-run ≥ 29 aa (~25% of tile)
  Final library:
    • proteins kept: 78
    • tiles kept   : 279
  Expected Stage-2 66aa tiles before Stage-2 filtering: 13,950

[Stage-2 (66aa) library]
  Params:
    • tile_len=66, stride=1
    • disorder_thr=0.5, truth_thr=0.65
    • dedupe: contiguous-run ≥ 17 aa (~25% of tile)
  Final library:
    • proteins kept: 67
    • tiles kept   : 222

[Per-protein counts] Stage-1 (115aa) tiles kept – top 10
uniprot_id
P46013    32
Q8NDV7    19
Q9UPQ9    17
Q5UIP0    15
Q9UQ35    12
Q13428    12
Q9C0C2    12
Q14671     7
Q14684     6
Q14978     6
Q71RC2     5
O15234     5
Q9NYF8     5
Q13427     5
Q13206     4
P46087     4
Q9BYJ9     4
Q7Z739     4
P19338     4
P18583     4

[Per-protein counts] Stage-2 (66aa) tiles kept – top 10
uniprot_id
P46013    28
Q8NDV7    19
Q9UPQ9    16
Q5UIP0    14
Q9C0C2    12
Q13428    10
Q14671     7
Q1468

Building changeable parameters for library size and proteins that are in it- omitting pml body for both